In [1]:
import pandas as pd
import numpy as np
from math import *
from sklearn.model_selection import train_test_split
from math import sqrt
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import itertools
from itertools import chain
from typing import Iterable, Sequence, Tuple
import time
pd.__version__

'0.18.1'

In [25]:
#log10()
from decimal import Decimal
x = 1E-1024
log10(31288.86183185)
#round(.0000000001,10)

4.49538976547482

In [2]:
## Read dataset
### sarthak path ->  "D:\\Affine_Projects\\WB_Warner_Bros\\MMM\\AD_Creation\\Lego_AD\\14062019"
### hemanth path -> "C:\\Users\\XHSINDHA\\Desktop\\WB\\Marketing\\MMM\\Models\\0617"
### path = "D:\\Affine_Projects\\WB_Warner_Bros\\MMM\\AD_Creation\\Lego_AD\\28062019"
path = "D:\\Affine_Projects\\WB_Warner_Bros\\MMM\\AD_Creation\\Lego_AD\\19072019"
raw_ad = pd.read_excel(path+"\\MMM_Lego_AD3.xlsx")
ad = raw_ad.sort_values(by=['Title','Week Number'])
print(ad.shape)
ad.head()

(605, 71)


Total Units  Total Spend  ASP Week Ending                 Title  \
0            0          0.0  0.0  2016-09-10  Cars-3-Driven-to-Win   
1            0          0.0  0.0  2016-09-17  Cars-3-Driven-to-Win   
2            0          0.0  0.0  2016-09-24  Cars-3-Driven-to-Win   
3            0          0.0  0.0  2016-10-01  Cars-3-Driven-to-Win   
4            0          0.0  0.0  2016-10-08  Cars-3-Driven-to-Win   

   Total Revenue  Sales Trend-Not Flatten  Sales Trend  Meta Critic Score  \
0            0.0                 0.000256     0.000256                 72   
1            0.0                 0.000311     0.000311                 72   
2            0.0                 0.002146     0.002146                 72   
3            0.0                 0.001160     0.001160                 72   
4            0.0                 0.000969     0.000969                 72   

   Metacritic number of user ratings        ...         pos_thanksgiving  \
0                                 25        ...                      0.0   
1                                 25        ...                      0.0   
2                                 25        ...                      0.0   
3                                 25        ...                      0.0   
4                                 25        ...                      0.0   

   pos_christmas  Facebook_1st week  Video_1st week  Search_1st week  \
0            0.0                0.0             0.0              0.0   
1            0.0                0.0             0.0              0.0   
2            0.0                0.0             0.0              0.0   
3            0.0                0.0             0.0              0.0   
4            0.0                0.0             0.0              0.0   

   Display_1st week  Facebook_2nd week  Video_2nd week  Search_2nd week  \
0               0.0                0.0             0.0              0.0   
1               0.0                0.0             0.0              0.0   
2               0.0                0.0             0.0              0.0   
3               0.0                0.0             0.0              0.0   
4               0.0                0.0             0.0              0.0   

   Display_2nd week  
0               0.0  
1               0.0  
2               0.0  
3               0.0  
4               0.0  

[5 rows x 71 columns]

### New Adstock Calculation

In [3]:
def roundoff_adstock(x):
    if(x==0):
        return 100
    else:
        log_10 = log10(x)
        floor_log10 = floor(log_10)
        lower_power10 = pow(10,floor_log10)
        first_digit = floor(x/lower_power10)
        final = first_digit * lower_power10
        return final


def add_single_adstock_col(df,col,decay_sat_par_val_dict,decay_param,saturation_param):
    df = df[[col]]
    decay = decay_sat_par_val_dict[decay_param]
    saturation = decay_sat_par_val_dict[saturation_param]
    df["pvals_"+col]=0
    df["adstock_"+col]=0
    for index in df.index:
        curr_val = df.loc[index,col]
        if(index==0):
            df.loc[index,"pvals_"+col] = 1- (1/exp((curr_val*saturation)/roundoff_adstock(curr_val)))
            df.loc[index,"adstock_"+col] = curr_val
        else:
            prev_pvals = df.loc[index-1,"pvals_"+col]
            df.loc[index,"pvals_"+col] = 1 - ((1-prev_pvals*exp(-1*decay))/exp((curr_val*saturation)/roundoff_adstock(curr_val)))
            df.loc[index,"adstock_"+col] = curr_val+(df.loc[index-1,"adstock_"+col]*df.loc[index,"pvals_"+col])
    df["ads_"+col+"_D"+str(decay_param)+"_S"+str(saturation_param)] = df["adstock_"+col]
    df = df.drop([col,"pvals_"+col,"adstock_"+col],axis=1)
    return df  

def get_adstock(df,adstock_cols,decay_min,decay_max,saturation_min,saturation_max):
    df2 = df.copy()
    for col in adstock_cols:
        #print("Column = "+col)
        decay_param = decay_min
        while(decay_param<=decay_max):
            saturation_param = saturation_min
            while(saturation_param<=saturation_max):
               # print("col=",col+" decay_param = "+str(decay_param)+" saturation_param= "+str(saturation_param))
                s_time = time.time()
                df1 = add_single_adstock_col(df[[col]],col,decay_sat_par_val_dict,decay_param,saturation_param)
                e_time = time.time()
               # print("time for process = ",str(e_time-s_time))
                colName = df1.columns[0]
                df2[colName] = df1[colName]
                saturation_param = saturation_param +1
            decay_param = decay_param+1
    return df2


In [4]:
traditional_cols = ['TVSpend','TradeDisplayPlacement','TradeDisplayPlacement_Combined']
digital_cols = ['Video','Search','POSRebate','POSRebate_Combined','TubularViews']
other_cols = ['Display','Facebook']
decay_sat_par_val_dict = {
    1:0.0625,
    2:0.125,
    3:0.25,
    4:0.5,
    5:1,
    6:1.5,
    7:2,
    8:2.5,
    9:0.3,
    10:0.4,
    11:0.6,
    12:0.7,
    13:0.8,
    14:1.2,
    15:1.4
}
traditional_saturation_min = 5
traditional_saturation_max = 8
traditional_decay_min = 1
traditional_decay_max = 3
digital_saturation_min = 1
digital_saturation_max = 5
digital_decay_min = 3
digital_decay_max = 5
other_saturation_min = 1 
other_saturation_max = 15
other_decay_min = 9
other_decay_max = 15

In [5]:
title_list = ad['Title'].unique()
df_traditional = pd.DataFrame()
df_digital = pd.DataFrame()
df_other = pd.DataFrame()
for title in title_list:
    ad_title = ad[ad['Title']==title]
    ad_title = ad_title.reset_index()
    df_traditional_title = get_adstock(ad_title,traditional_cols,traditional_decay_min,traditional_decay_max,traditional_saturation_min,traditional_saturation_max)
    df_traditional = df_traditional.append(df_traditional_title,ignore_index=True)
    df_digital_title = get_adstock(ad_title,digital_cols,digital_decay_min,digital_decay_max,digital_saturation_min,digital_saturation_max)
    df_digital = df_digital.append(df_digital_title,ignore_index=True)
    df_other_title = get_adstock(ad_title,other_cols,other_decay_min,other_decay_max,other_saturation_min,other_saturation_max)
    df_other = df_other.append(df_other_title,ignore_index=True)


In [6]:
df_traditional = df_traditional.drop(['index'],axis=1)
df_digital = df_digital.drop(['index'],axis=1)
df_other = df_other.drop(['index'],axis=1)

In [7]:
## Combining Traditional and Digital to get a single AD. Dropping some cols from digital 
## which are already present in traditional so as to merge properly

digital_cols_drop = ['Total Units','ASP','Total Spend','Week Ending','Total Revenue','Sales Trend-Not Flatten','Sales Trend','Meta Critic Score','Metacritic number of user ratings','Is Action',
                    'Is Adventure', 'Is Open World', 'Is Fighting', 'Is Other','Physical ASP', 'Digital ASP','Thanksgiving Flag','Christmas Flag', 
                     '1st week', '2nd week', 'black Friday week number','Install base', 'same genre', 'different genre', 'TVSpend', 'Facebook',
                     'Video','Search', 'Display','POSRebate','TradeDisplayPlacement','POSRebate_Combined','TradeDisplayPlacement_Combined','TubularViews','Calender Week','Month','Games Developed Before this game',
                    'Total Games','Games with WB as Publisher','First Game of Studio(Years Ago)','Average Metacritic Score','Average Revenue',
                    'No of prequels from franchise','Last Game of Franchise','Average Gap in Game Release(Years)','Jan','Feb','Mar','Apr','May','Jun','Jul',
                     'Aug','Sep','Oct','Nov','Dec','pos_thanksgiving','pos_christmas','Facebook_1st week','Video_1st week','Search_1st week',
                    'Display_1st week','Facebook_2nd week','Video_2nd week','Search_2nd week','Display_2nd week']
other_cols_drop = digital_cols_drop
df_ads_merge = pd.merge(df_traditional,df_digital.drop(digital_cols_drop,axis=1),how='inner',on=['Title','Week Number','Week Number New'])
df_ads_merge = pd.merge(df_ads_merge,df_other.drop(other_cols_drop,axis=1),how='inner',on=['Title','Week Number','Week Number New'])
print(df_ads_merge.columns)
df_ads_merge.to_excel(path+"\\adstock_file.xlsx")
df_ads_merge.head()

Index(['Total Units', 'Total Spend', 'ASP', 'Week Ending', 'Title',
       'Total Revenue', 'Sales Trend-Not Flatten', 'Sales Trend',
       'Meta Critic Score', 'Metacritic number of user ratings',
       ...
       'ads_Facebook_D15_S6', 'ads_Facebook_D15_S7', 'ads_Facebook_D15_S8',
       'ads_Facebook_D15_S9', 'ads_Facebook_D15_S10', 'ads_Facebook_D15_S11',
       'ads_Facebook_D15_S12', 'ads_Facebook_D15_S13', 'ads_Facebook_D15_S14',
       'ads_Facebook_D15_S15'],
      dtype='object', length=394)


Total Units  Total Spend  ASP Week Ending                 Title  \
0            0          0.0  0.0  2016-09-10  Cars-3-Driven-to-Win   
1            0          0.0  0.0  2016-09-17  Cars-3-Driven-to-Win   
2            0          0.0  0.0  2016-09-24  Cars-3-Driven-to-Win   
3            0          0.0  0.0  2016-10-01  Cars-3-Driven-to-Win   
4            0          0.0  0.0  2016-10-08  Cars-3-Driven-to-Win   

   Total Revenue  Sales Trend-Not Flatten  Sales Trend  Meta Critic Score  \
0            0.0                 0.000256     0.000256                 72   
1            0.0                 0.000311     0.000311                 72   
2            0.0                 0.002146     0.002146                 72   
3            0.0                 0.001160     0.001160                 72   
4            0.0                 0.000969     0.000969                 72   

   Metacritic number of user ratings          ...           \
0                                 25          ...            
1                                 25          ...            
2                                 25          ...            
3                                 25          ...            
4                                 25          ...            

   ads_Facebook_D15_S6  ads_Facebook_D15_S7  ads_Facebook_D15_S8  \
0                  0.0                  0.0                  0.0   
1                  0.0                  0.0                  0.0   
2                  0.0                  0.0                  0.0   
3                  0.0                  0.0                  0.0   
4                  0.0                  0.0                  0.0   

   ads_Facebook_D15_S9  ads_Facebook_D15_S10  ads_Facebook_D15_S11  \
0                  0.0                   0.0                   0.0   
1                  0.0                   0.0                   0.0   
2                  0.0                   0.0                   0.0   
3                  0.0                   0.0                   0.0   
4                  0.0                   0.0                   0.0   

   ads_Facebook_D15_S12  ads_Facebook_D15_S13  ads_Facebook_D15_S14  \
0                   0.0                   0.0                   0.0   
1                   0.0                   0.0                   0.0   
2                   0.0                   0.0                   0.0   
3                   0.0                   0.0                   0.0   
4                   0.0                   0.0                   0.0   

   ads_Facebook_D15_S15  
0                   0.0  
1                   0.0  
2                   0.0  
3                   0.0  
4                   0.0  

[5 rows x 394 columns]

## For Capping

In [8]:
### For Capping the high low values. Uncomment df_ads_merge only if capping is required

def capped(row,col,lower):
    if(row[col]<lower):
        return lower
#     elif(row[col]>upper):
#         return upper
    else:
        return row[col]
def get_capped_df(df,col,lower):
    df[col] = df.apply(lambda row:capped(row,col,lower),axis=1)
    return df

df_ads_capped = get_capped_df(df_ads_merge.copy(),'Total Units',1000)
#df_ads_capped = get_capped_df(df_ads_capped.copy(),'TubularViews',123559,6855385) 
df_ads_merge = df_ads_capped

df_ads_merge 

Total Units   Total Spend        ASP Week Ending  \
0           1000      0.000000   0.000000  2016-09-10   
1           1000      0.000000   0.000000  2016-09-17   
2           1000      0.000000   0.000000  2016-09-24   
3           1000      0.000000   0.000000  2016-10-01   
4           1000      0.000000   0.000000  2016-10-08   
5           1000      0.000000   0.000000  2016-10-15   
6           1000      0.000000   0.000000  2016-10-22   
7           1000      0.000000   0.000000  2016-10-29   
8           1000      0.000000   0.000000  2016-11-05   
9           1000      0.000000   0.000000  2016-11-12   
10          1000      0.000000   0.000000  2016-11-19   
11          1000      0.000000   0.000000  2016-11-26   
12          1000      0.000000   0.000000  2016-12-03   
13          1000      0.000000   0.000000  2016-12-10   
14          1000      0.000000   0.000000  2016-12-17   
15          1000      0.000000   0.000000  2016-12-24   
16          1000      0.000000   0.000000  2016-12-31   
17          1000      0.000000   0.000000  2017-01-07   
18          1000      0.000000   0.000000  2017-01-14   
19          1000      0.000000   0.000000  2017-01-21   
20          1000      0.000000   0.000000  2017-01-28   
21          1000      0.000000   0.000000  2017-02-04   
22          1000      0.000000   0.000000  2017-02-11   
23          1000      0.000000   0.000000  2017-02-18   
24          1000      0.000000   0.000000  2017-02-25   
25          1000      0.000000   0.000000  2017-03-04   
26          1000      0.000000   0.000000  2017-03-11   
27          1000      0.000000   0.000000  2017-03-18   
28          1000      0.000000   0.000000  2017-03-25   
29          1000    168.659341   0.000000  2017-04-01   
..           ...           ...        ...         ...   
575         1121    835.739130  30.671512  2018-08-18   
576         1000    835.739130  30.535732  2018-08-25   
577         1000    835.739130  25.673462  2018-09-01   
578         1011    835.739130  25.619814  2018-09-08   
579         1000    835.739130  25.482957  2018-09-15   
580         1000    835.739130  23.578490  2018-09-22   
581         1189    835.739130  22.513835  2018-09-29   
582         1000    835.739130  22.717625  2018-10-06   
583         1000    835.739130  23.008238  2018-10-13   
584         1000    835.739130  23.555934  2018-10-20   
585         1000    835.739130  23.227433  2018-10-27   
586         1128    835.739130  21.790425  2018-11-03   
587         1000    358.173913  21.685475  2018-11-10   
588         1036      0.000000  18.526182  2018-11-17   
589         4380  16184.800000  20.264036  2018-11-24   
590         2484   1371.200000  20.164783  2018-12-01   
591         1543      0.000000  21.687082  2018-12-08   
592         2463   7626.285714  21.109935  2018-12-15   
593         5116  13346.000000  20.724122  2018-12-22   
594         4765   7776.857143  21.242871  2018-12-29   
595         2215   1542.857143  23.056933  2019-01-05   
596         1057      0.000000  22.971233  2019-01-12   
597         1000      0.000000  23.154531  2019-01-19   
598         1000      0.000000  23.850252  2019-01-26   
599         1000      0.000000  21.754943  2019-02-02   
600         1000      0.000000  21.927333  2019-02-09   
601         1000      0.000000  21.971815  2019-02-16   
602         1000    106.909091  22.331680  2019-02-23   
603         1089   3663.163636  21.795770  2019-03-02   
604         1000   1569.927273  21.972032  2019-03-09   

                                           Title  Total Revenue  \
0                           Cars-3-Driven-to-Win       0.000000   
1                           Cars-3-Driven-to-Win       0.000000   
2                           Cars-3-Driven-to-Win       0.000000   
3                           Cars-3-Driven-to-Win       0.000000   
4                           Cars-3-Driven-to-Win       0.000000   
5                           Cars-3-Driven-to-Win       0.000000   
6

In [9]:
## Function to calculate natural log of the columns.
def get_ln_values(df,col):
    df1 = df.copy()
    df1["ln_"+col] = 0
    for index in df.index:
        value = df.loc[index,col]
        if(value>0):
            cal = log(value)
            if(cal>0):
                df1.loc[index,"ln_"+col] = cal
    return df1

all_columns = df_ads_merge.columns
not_ln_columns = ['Week Ending','Title','Total Revenue','Total Spend','Sales Trend','Meta Critic Score','Metacritic number of user ratings',
                 'Is Action','Is Adventure','Is Open World','Is Fighting','Is Other','Thanksgiving Flag','Christmas Flag','1st week','2nd week','black Friday week number','Install base','same genre',
                 'different genre','Calender Week','Month','Games Developed Before this game','Total Games',
                'Games with WB as Publisher','First Game of Studio(Years Ago)','Average Metacritic Score','Average Revenue',
                'No of prequels from franchise','Last Game of Franchise','Average Gap in Game Release(Years)','Jan','Feb','Mar','Apr','May','Jun','Jul',
                  'Aug','Sep','Oct','Nov','Dec','pos_thanksgiving','pos_christmas','Facebook_1st week','Video_1st week','Search_1st week',
                    'Display_1st week','Facebook_2nd week','Video_2nd week','Search_2nd week','Display_2nd week']
## 'TVSpend','Facebook','Video','Search','Display','POSRebate','TradeDisplayPlacement','OtherTradeSpend','TubularViews'
ln_columns = np.setdiff1d(all_columns,not_ln_columns,assume_unique=True)
df_ads = df_ads_merge.copy()
for col in ln_columns:
    df1 = get_ln_values(df_ads_merge.copy()[[col]],col)
    ln_column_name = df1.columns[1]
    df_ads[ln_column_name] = df1[ln_column_name]

df_ads.to_excel(path+"\\ln_adstock_file.xlsx")    
df_ads.head()


Total Units  Total Spend  ASP Week Ending                 Title  \
0         1000          0.0  0.0  2016-09-10  Cars-3-Driven-to-Win   
1         1000          0.0  0.0  2016-09-17  Cars-3-Driven-to-Win   
2         1000          0.0  0.0  2016-09-24  Cars-3-Driven-to-Win   
3         1000          0.0  0.0  2016-10-01  Cars-3-Driven-to-Win   
4         1000          0.0  0.0  2016-10-08  Cars-3-Driven-to-Win   

   Total Revenue  Sales Trend-Not Flatten  Sales Trend  Meta Critic Score  \
0            0.0                 0.000256     0.000256                 72   
1            0.0                 0.000311     0.000311                 72   
2            0.0                 0.002146     0.002146                 72   
3            0.0                 0.001160     0.001160                 72   
4            0.0                 0.000969     0.000969                 72   

   Metacritic number of user ratings           ...             \
0                                 25           ...              
1                                 25           ...              
2                                 25           ...              
3                                 25           ...              
4                                 25           ...              

   ln_ads_Facebook_D15_S6  ln_ads_Facebook_D15_S7  ln_ads_Facebook_D15_S8  \
0                     0.0                     0.0                     0.0   
1                     0.0                     0.0                     0.0   
2                     0.0                     0.0                     0.0   
3                     0.0                     0.0                     0.0   
4                     0.0                     0.0                     0.0   

   ln_ads_Facebook_D15_S9  ln_ads_Facebook_D15_S10  ln_ads_Facebook_D15_S11  \
0                     0.0                      0.0                      0.0   
1                     0.0                      0.0                      0.0   
2                     0.0                      0.0                      0.0   
3                     0.0                      0.0                      0.0   
4                     0.0                      0.0                      0.0   

   ln_ads_Facebook_D15_S12  ln_ads_Facebook_D15_S13  ln_ads_Facebook_D15_S14  \
0                      0.0                      0.0                      0.0   
1                      0.0                      0.0                      0.0   
2                      0.0                      0.0                      0.0   
3                      0.0                      0.0                      0.0   
4                      0.0                      0.0                      0.0   

   ln_ads_Facebook_D15_S15  
0                      0.0  
1                      0.0  
2                      0.0  
3                      0.0  
4                      0.0  

[5 rows x 735 columns]

In [374]:
### For Finding Correlation
def get_Correlation(df1,col):
    columns= df1.columns
    col_list = ['Total Units','ln_Total Units']
    for column in columns:
        if(column.find(col)>=0):
            col_list.append(column)
            
    
    df = df1[col_list].copy()
    df = df[df['Total Units']!=0]
    df_corr = df.corr()
    df_corr_reset = df_corr[['Total Units','ln_Total Units']].reset_index()
    df_corr_reset.to_excel(path+"\\corr\\"+col+".xlsx")
    return df_corr_reset
corr_columns = ['TVSpend','Facebook','Video','Search','Display','POSRebate','TradeDisplayPlacement',\
                'POSRebate_Combined','TradeDisplayPlacement_Combined','TubularViews'] 
for column in corr_columns:
    corr= get_Correlation(df_ads,column)


PermissionError: [Errno 13] Permission denied: 'D:\\Affine_Projects\\WB_Warner_Bros\\MMM\\AD_Creation\\Lego_AD\\19072019\\corr\\Display.xlsx'

# Column Selection for AD

In [98]:
## top 2 adstock vars for each marketing channel-  manually selected from correlation
## Tubular views has been commented as we have added it in fixed cols and removed from decision vars.

tv_spend = ['ln_ads_TVSpend_D1_S7','ln_ads_TVSpend_D1_S8']
facebook = ['ln_ads_Facebook_D3_S4','ln_ads_Facebook_D3_S5']
#video = ['ln_VideoSpend']
video = ['ln_ads_Video_D3_S4', 'ln_ads_Video_D3_S5']
##   'ln_VideoImpressions' -> d3s3 and d3s4
search = ['ln_ads_Search_D3_S4','ln_ads_Search_D3_S5']
display = ['ln_ads_Display_D3_S4','ln_ads_Display_D3_S5']
pos_rebate = ['ln_ads_POSRebate_D2_S5','ln_ads_POSRebate_D2_S6']
trade_display_repayment = ['ln_ads_TradeDisplayPlacement_D1_S5','ln_ads_TradeDisplayPlacement_D1_S6']

#other_trade_spend = ['ln_ads_OtherTradeSpend_D3_S5','ln_ads_OtherTradeSpend_D3_S6']

#tubular_views = ['ln_TubularViews','ln_ads_TubularViews_D5_S1']



# 'Install base','Is Other','Is Fighting','Week Number' ,'Meta Critic Score',
#fixed_cols = ['Week Ending','Title','Sales Trend','Metacritic number of user ratings','Physical ASP','Digital ASP','Install base','Is Other','Is Fighting','Is Action','Is Adventure','Is Open World','Week Number','Thanksgiving Flag','Christmas Flag','1st week','2nd week','black Friday week number','same genre','different genre','ln_Total Units']
#fixed_cols = ['Week Ending','Title','ASP','Thanksgiving Flag','same genre','different genre','Metacritic number of user ratings','ln_Total Units']
#['ASP','Sales Trend','Week Number','1st week','2nd week','TubularViews', 'Christmas Flag','Is Action','Is Adventure','Is Open World','Thanksgiving Flag','same genre','different genre','black Friday week number','Metacritic number of user ratings']

fixed_cols = ['Week Ending','Title','ASP','Sales Trend','Week Number','Meta Critic Score','1st week','2nd week','Christmas Flag','Is Action','Is Adventure','Is Open World','Thanksgiving Flag','same genre','different genre','black Friday week number','TubularViews','ln_Total Units']

In [100]:
## creating all the possible 1024 combinations
list_of_lists = [tv_spend,facebook,video,display,search,pos_rebate,trade_display_repayment]
# facebook,display,search,video
# tv_spend,facebook,search,display,video,pos_rebate,trade_display_repayment,other_trade_spend
# tubular_views
iterations_vector = list(itertools.product(*list_of_lists))

#converting list of tuples to list of lists
iterations_list = [list(elem) for elem in iterations_vector]
len(iterations_list)

128

In [101]:
## creating a list of column names that form 1024 dataframes based on the 1024 combinations above
all_train_cols_list = []

for i in range(0, (len(iterations_list))):
    #print(i)
    x = list(fixed_cols)
    x.extend(iterations_list[i])
    all_train_cols_list.append(x)  
len(all_train_cols_list)

128

In [102]:
print(len(all_train_cols_list[0]))
print(all_train_cols_list[0])

25
['Week Ending', 'Title', 'ASP', 'Sales Trend', 'Week Number', 'Meta Critic Score', '1st week', '2nd week', 'Christmas Flag', 'Is Action', 'Is Adventure', 'Is Open World', 'Thanksgiving Flag', 'same genre', 'different genre', 'black Friday week number', 'TubularViews', 'ln_Total Units', 'ln_ads_TVSpend_D1_S7', 'ln_ads_Facebook_D3_S4', 'ln_ads_Video_D3_S4', 'ln_ads_Display_D3_S4', 'ln_ads_Search_D3_S4', 'ln_ads_POSRebate_D2_S5', 'ln_ads_TradeDisplayPlacement_D1_S5']


In [103]:
## Creating a list of 512 dataframes that we need to train the model on
all_train_dfs = []

for i in range(0, len(all_train_cols_list)):
    all_train_dfs.append(df_ads[all_train_cols_list[i]])

len(all_train_dfs)

128

In [104]:
all_train_dfs[0]

Week Ending                                        Title        ASP  \
0    2016-09-10                         Cars-3-Driven-to-Win   0.000000   
1    2016-09-17                         Cars-3-Driven-to-Win   0.000000   
2    2016-09-24                         Cars-3-Driven-to-Win   0.000000   
3    2016-10-01                         Cars-3-Driven-to-Win   0.000000   
4    2016-10-08                         Cars-3-Driven-to-Win   0.000000   
5    2016-10-15                         Cars-3-Driven-to-Win   0.000000   
6    2016-10-22                         Cars-3-Driven-to-Win   0.000000   
7    2016-10-29                         Cars-3-Driven-to-Win   0.000000   
8    2016-11-05                         Cars-3-Driven-to-Win   0.000000   
9    2016-11-12                         Cars-3-Driven-to-Win   0.000000   
10   2016-11-19                         Cars-3-Driven-to-Win   0.000000   
11   2016-11-26                         Cars-3-Driven-to-Win   0.000000   
12   2016-12-03                         Cars-3-Driven-to-Win   0.000000   
13   2016-12-10                         Cars-3-Driven-to-Win   0.000000   
14   2016-12-17                         Cars-3-Driven-to-Win   0.000000   
15   2016-12-24                         Cars-3-Driven-to-Win   0.000000   
16   2016-12-31                         Cars-3-Driven-to-Win   0.000000   
17   2017-01-07                         Cars-3-Driven-to-Win   0.000000   
18   2017-01-14                         Cars-3-Driven-to-Win   0.000000   
19   2017-01-21                         Cars-3-Driven-to-Win   0.000000   
20   2017-01-28                         Cars-3-Driven-to-Win   0.000000   
21   2017-02-04                         Cars-3-Driven-to-Win   0.000000   
22   2017-02-11                         Cars-3-Driven-to-Win   0.000000   
23   2017-02-18                         Cars-3-Driven-to-Win   0.000000   
24   2017-02-25                         Cars-3-Driven-to-Win   0.000000   
25   2017-03-04                         Cars-3-Driven-to-Win   0.000000   
26   2017-03-11                         Cars-3-Driven-to-Win   0.000000   
27   2017-03-18                         Cars-3-Driven-to-Win   0.000000   
28   2017-03-25                         Cars-3-Driven-to-Win   0.000000   
29   2017-04-01                         Cars-3-Driven-to-Win   0.000000   
..          ...                                          ...        ...   
621  2018-08-18  Scribblenauts Showdown/Scribblenauts Fiesta  30.671512   
622  2018-08-25  Scribblenauts Showdown/Scribblenauts Fiesta  30.535732   
623  2018-09-01  Scribblenauts Showdown/Scribblenauts Fiesta  25.673462   
624  2018-09-08  Scribblenauts Showdown/Scribblenauts Fiesta  25.619814   
625  2018-09-15  Scribblenauts Showdown/Scribblenauts Fiesta  25.482957   
626  2018-09-22  Scribblenauts Showdown/Scribblenauts Fiesta  23.578490   
627  2018-09-29  Scribblenauts Showdown/Scribblenauts Fiesta  22.513835   
628  2018-10-06  Scribblenauts Showdown/Scribblenauts Fiesta  22.717625   
629  2018-10-13  Scribblenauts Showdown/Scribblenauts Fiesta  23.008238   
630  2018-10-20  Scribblenauts Showdown/Scribblenauts Fiesta  23.555934   
631  2018-10-27  Scribblenauts Showdown/Scribblenauts Fiesta  23.227433   
632  2018-11-03  Scribblenauts Showdown/Scribblenauts Fiesta  21.790425   
633  2018-11-10  Scribblenauts Showdown/Scribblenauts Fiesta  21.685475   
634  2018-11-17  Scribblenauts Showdown/Scribblenauts Fiesta  18.526182   
635  2018-11-24  Scribblenauts Showdown/Scribblenauts Fiesta  20.264036   
636  2018-12-01  Scribblenauts Showdown/Scribblenauts Fiesta  20.164783   
637  2018-12-08  Scribblenauts Showdown/Scribblenauts Fiesta  21.687082   
638  2018-12-15  Scribblenauts Showdown/Scribblenauts Fiesta  21.109935   
639  2018-12-22  Scribblenauts Showdown/Scribblenauts Fiesta  20.724122   
640  2018-12-29  Scribblenauts Showdown/Scribblenauts Fiesta  21.242871   
641  2019-01-05  Scribblenauts Showdown/Scribblenauts Fiesta  23.056933   
642  2019-01-12  Scribblenaut

In [105]:
# linear regression for the 1024 dataframes. Select the best combination based on least MAPE or highest Adjusted Rsquared

# For the best model, calculate the %contribution by each channel


In [10]:
# function for splitting train test dataset based on games or randomly
from sklearn.utils import shuffle

def getTrainTestGames(df_X,df_Y,testGames):
    X_test = df_X[df_X['Title'].isin(testGames)]
    X_train = df_X[~df_X['Title'].isin(testGames)]
    Y_test = df_Y[df_Y.index.isin(X_test.index)]
    Y_train = df_Y[df_Y.index.isin(X_train.index)]
    return X_train,X_test,Y_train,Y_test

def getSplitTrainTest(df_X,df_Y,testSize):
    X_train,X_test,Y_train,Y_test = train_test_split(df_X,df_Y,test_size=testSize,random_state=7)
    return (X_train,X_test,Y_train,Y_test)

### Function for scaling the data
from sklearn import preprocessing
def getScaledData(X):
    #scaler = preprocessing.StandardScaler().fit(X)
    X1 = X.copy().drop(['Week Ending','Title'],axis=1)
    scaler = preprocessing.MinMaxScaler().fit(X1)
    scaled_arr = scaler.transform(X1) 
    df_AD_X1_scaled=  pd.DataFrame(scaled_arr,columns =X1.columns)
    df = pd.concat([X[['Week Ending','Title']],df_AD_X1_scaled],axis=1)
    # (X,Y,scaler)
    # (df,scaler)
    return (X,scaler)

### Function to create LR model
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant
import statsmodels.api as sm
def getLinearModel(X_train,Y_train):
    lr_model_ols = sm.OLS(Y_train.values,X_train.values).fit()
    return (lr_model_ols,X_train.columns)  

### Calculate Various Metrics
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
def Rsquare(yoriginal,ypred):
    SS_Residual = sum((yoriginal-ypred)**2)
    SS_Total = sum((yoriginal-np.mean(yoriginal))**2)
    r_squared = 1 - (float(SS_Residual))/SS_Total
    return r_squared
def adjRsquare(yoriginal,ypred,numIndVar):
    SS_Residual = sum((yoriginal-ypred)**2)
    SS_Total = sum((yoriginal-np.mean(yoriginal))**2)
    r_squared = 1 - (float(SS_Residual))/SS_Total
    adjusted_r_squared = 1 - (1-r_squared)*(len(yoriginal)-1)/(len(yoriginal)-numIndVar-1)
    return adjusted_r_squared
def replace_to_percent(df_add,cols):
    df_add['Total Spend'] = df_add[list(cols)].sum(axis=1)   
    for col in cols:
        df_add[col] = (df_add[col] /  df_add['Total Spend'])*100
    return df_add
def w_mape(y_true,y_pred):
    abs_error_percent = np.abs((y_true- y_pred)/y_true)*100
    return (sum(y_true*abs_error_percent)/sum(y_true))

    


In [11]:
def getPreProcessData(df,colsForModel,nonMarketingCols,marketingLnCols,testGames):
    dict_preprocess_data = {}
    dfAds = df
    dfAds = dfAds[dfAds['ln_Total Units']!=0]
    dfAdsPruned = dfAds.dropna()
    
    dfAdsPruned = shuffle(dfAdsPruned,random_state=7)
    df_Y = dfAdsPruned['ln_Total Units']
    df_X = dfAdsPruned.drop(['ln_Total Units'],axis=1)
    
    X_scaled,scaler = getScaledData(df_X)
    X_train,X_test,Y_train,Y_test = getTrainTestGames(X_scaled,df_Y,testGames)
    #X_train,X_test,Y_train,Y_test = getSplitTrainTest(X_scaled,df_Y,.2)
    dict_preprocess_data['X_train_all']= X_train
    dict_preprocess_data['X_test_all'] = X_test
    X_train = X_train.drop(['Week Ending','Title'],axis=1)
    X_test = X_test.drop(['Week Ending','Title'],axis=1)
    XTrainScaledConst = X_train[colsForModel]
    XTestScaledConst = X_test[colsForModel]
    XTrainScaledConst = sm.add_constant(XTrainScaledConst)
    XTestScaledConst = sm.add_constant(XTestScaledConst,has_constant='add')
    dict_preprocess_data['Y_train'] = Y_train
    dict_preprocess_data['Y_test']= Y_test
    dict_preprocess_data['XTrainScaledConst']= XTrainScaledConst
    dict_preprocess_data['XTestScaledConst']= XTestScaledConst
    dict_preprocess_data['scaler'] = scaler
    return dict_preprocess_data    

In [12]:
def calculateLinearRegression(df_overall,colsForModel,nonMarketingCols,marketingLnCols,testGames):
    dict_preprocess_data = getPreProcessData(df_overall,colsForModel,nonMarketingCols,marketingLnCols,testGames)
    XTrainScaledConst = dict_preprocess_data['XTrainScaledConst']
    XTrainScaledConst = XTrainScaledConst
    XTestScaledConst = dict_preprocess_data['XTestScaledConst']
    XTestScaledConst = XTestScaledConst
    X_train_all = dict_preprocess_data['X_train_all']
    X_test_all = dict_preprocess_data['X_test_all']
    Y_train = dict_preprocess_data['Y_train']
    Y_test = dict_preprocess_data['Y_test']
    scaler = dict_preprocess_data['scaler']
    lr_model_ols,column_names = getLinearModel(XTrainScaledConst,Y_train)
    model_param_dict = getModelParamDict(lr_model_ols,column_names,XTrainScaledConst,XTestScaledConst,X_train_all,X_test_all,Y_train,Y_test,scaler)
    return model_param_dict

In [13]:
def getModelParamDict(lr_model_ols,column_names,XTrainScaledConst,XTestScaledConst,X_train_all,X_test_all,Y_train,Y_test,scaler):
    model_dict = {}
    model_dict["lr_model_ols"] = lr_model_ols
    model_dict["column_names"] = column_names
    model_dict["XTrainScaledConst"] = XTrainScaledConst
    model_dict["XTestScaledConst"] = XTestScaledConst
    model_dict["X_train_all"] = X_train_all
    model_dict["X_test_all"] = X_test_all
    model_dict["Y_train"] = Y_train
    model_dict["Y_test"] = Y_test
    model_dict["scaler"] = scaler
    #print(model_dict["column_names"])
    return model_dict

In [68]:
all_train_dfs[0].corr().to_excel(path+"\\corr_0_video.xlsx")

In [114]:
### Calculate Linear Regression models for all the combinations 
count=0
df_metrics = pd.DataFrame(columns = ['test_mape','train_mape','Rsquare','Adj-Rsquare'])
df_model_params ={}
for df_Ads in all_train_dfs:
    model_dict = calculateLinearRegression(df_Ads)
    df_model_params[count]=model_dict
    lr_model_ols = model_dict["lr_model_ols"]
    XTrainScaledConst = model_dict["XTrainScaledConst"]
    XTestScaledConst = model_dict["XTestScaledConst"]
    Y_train = model_dict["Y_train"]
    Y_test = model_dict["Y_test"]
    ypredict_train = lr_model_ols.predict(XTrainScaledConst)
    ypredict_test = lr_model_ols.predict(XTestScaledConst)
    train_mape = mean_absolute_percentage_error(Y_train,ypredict_train.values)
    test_mape = mean_absolute_percentage_error(Y_test,ypredict_test.values)
    train_rsquare =  Rsquare(Y_train,ypredict_train.values)
    train_adjrsquare = adjRsquare(Y_train,ypredict_train.values,XTrainScaledConst.shape[1]-1)
    df_metrics.loc[count] = [test_mape,train_mape,train_rsquare,train_adjrsquare]
    print(count)
    count = count+1


TypeError: calculateLinearRegression() missing 2 required positional arguments: 'marketing_original_cols' and 'testGames'

In [115]:
df_metrics.head()

Empty DataFrame
Columns: [test_mape, train_mape, Rsquare, Adj-Rsquare]
Index: []

In [116]:
df_metrics_sorted = df_metrics.sort_values(by=['test_mape','train_mape'])

In [117]:
df_metrics_sorted.head()

Empty DataFrame
Columns: [test_mape, train_mape, Rsquare, Adj-Rsquare]
Index: []

In [82]:
lr_model_ols = df_model_params[0]["lr_model_ols"]
col_name = df_model_params[0]["column_names"]
#col_name.values
lr_model_ols.summary(xname = list(col_name))

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.820
Model:                            OLS   Adj. R-squared:                  0.809
Method:                 Least Squares   F-statistic:                     76.48
Date:                Wed, 19 Jun 2019   Prob (F-statistic):          8.09e-123
Time:                        17:53:37   Log-Likelihood:                -506.35
No. Observations:                 392   AIC:                             1059.
Df Residuals:                     369   BIC:                             1150.
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
const                                  7.5767      0.046    165.290      0.000       7.487       7.667
ASP                                   -0.7564      0.091     -8.309      0.000      -0.935      -0.577
Sales Trend                            0.6910      0.116      5.934      0.000       0.462       0.920
Week Number                            0.4258      0.091      4.654      0.000       0.246       0.606
Meta Critic Score                      0.4238      0.101      4.210      0.000       0.226       0.622
1st week                               0.0081      0.062      0.129      0.897      -0.114       0.131
2nd week                               0.0264      0.061      0.431      0.667      -0.094       0.147
Christmas Flag                         0.1252      0.047      2.649      0.008       0.032       0.218
Is Action                              0.1023      0.065      1.585      0.114      -0.025       0.229
Is Adventure                           0.0733      0.072      1.016      0.310      -0.069       0.215
Is Open World                          0.0238      0.055      0.434      0.665      -0.084       0.131
Thanksgiving Flag                      0.0358      0.059      0.602      0.547      -0.081       0.153
same genre                             0.0307      0.047      0.647      0.518      -0.063       0.124
different genre                        0.0400      0.048      0.834      0.405      -0.054       0.134
black Friday week number               0.1641      0.058      2.807      0.005       0.049       0.279
TubularViews                           0.1300      0.097      1.343      0.180      -0.060       0.320
ln_ads_TVSpend_D1_S7                   0.3387      0.087      3.914      0.000       0.169       0.509
ln_ads_Facebook_D3_S4                 -0.0419      0.083     -0.505      0.614      -0.205       0.121
ln_ads_Video_D3_S4                     0.1870      0.105      1.783      0.075      -0.019       0.393
ln_ads_Display_D3_S4                  -0.0775      0.084     -0.926      0.355      -0.242       0.087
ln_ads_Search_D3_S4                   -0.0301      0.080     -0.377      0.707      -0.187       0.127
ln_ads_POSRebate_D2_S5                 0.7971      0.105      7.558      0.000       0.590       1.004
ln_ads_TradeDisplayPlacement_D1_S5     0.1446      0.074      1.961      0.051      -0.000       0.290
==============================================================================
Omnibus:                       17.601   Durbin-Watson:                   1.962
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               31.936
Skew:                          -0.267   Prob(JB):                     1.16e-07
Kurtosis:                       4.292   Cond. No.                         7.05
==========

In [81]:
### To check if we have any model in which marketing variable coefficients are coming to be all positve. It is needed for experimentaion
correct_model_num = []
for model_num in df_model_params:
    model_params_dict = df_model_params[model_num]
    lr_model_ols = model_params_dict["lr_model_ols"]
    betaCoef = lr_model_ols.params
#     if((betaCoef[7]>=0)  ):
#         correct_model_num.append(model_num)
    # & (betaCoef[9]>=0)  & (betaCoef[10]>=0) &(betaCoef[11]>=0)  & (betaCoef[12]>=0) & (betaCoef[13]>=0) 
    if(    (betaCoef[16]>=0)    & (betaCoef[18]>=0)  & (betaCoef[21]>=0) & (betaCoef[22]>=0) ):
        correct_model_num.append(model_num)
print(len(correct_model_num))
print(correct_model_num)
dfs = df_metrics_sorted.loc[correct_model_num]
dfs.sort_values(by=['test_mape','train_mape'])

128
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127]


test_mape  train_mape   Rsquare  Adj-Rsquare
94   10.829865   10.353723  0.819318     0.808546
90   10.830718   10.358733  0.819299     0.808526
30   10.832576   10.353727  0.819333     0.808561
26   10.833422   10.358764  0.819313     0.808541
126  10.833949   10.353634  0.819310     0.808537
92   10.834666   10.306863  0.820136     0.809413
122  10.834950   10.358778  0.819291     0.808517
88   10.835892   10.311881  0.820114     0.809390
62   10.836654   10.353680  0.819324     0.808552
86   10.837039   10.354114  0.819427     0.808662
28   10.837555   10.306966  0.820152     0.809429
58   10.837649   10.358819  0.819305     0.808532
82   10.837905   10.359479  0.819404     0.808636
124  10.838622   10.306582  0.820130     0.809406
24   10.838773   10.311960  0.820130     0.809406
22   10.839691   10.354166  0.819442     0.808677
120  10.839853   10.311461  0.820109     0.809383
18   10.840549   10.359528  0.819418     0.808651
74   10.841004   10.357464  0.819181     0.808401
118  10.841237   10.354266  0.819420     0.808654
60   10.841357   10.306602  0.820146     0.809423
78   10.842112   10.353128  0.819196     0.808416
84   10.842279   10.306761  0.820247     0.809530
114  10.842280   10.359660  0.819396     0.808629
56   10.842737   10.311564  0.820124     0.809400
80   10.843541   10.311948  0.820220     0.809501
10   10.843786   10.357390  0.819196     0.808417
54   10.843883   10.354326  0.819434     0.808669
14   10.844634   10.353059  0.819211     0.808432
50   10.844920   10.359715  0.819411     0.808644
..         ...         ...       ...          ...
9    10.880335   10.322661  0.819421     0.808655
21   10.880401   10.318124  0.819661     0.808910
117  10.880429   10.318090  0.819643     0.808890
105  10.880593   10.321908  0.819395     0.808628
43   10.880901   10.373002  0.818523     0.807703
47   10.881408   10.369189  0.818535     0.807716
17   10.881730   10.323539  0.819638     0.808884
113  10.881754   10.323560  0.819619     0.808865
55   10.881903   10.371829  0.818764     0.807959
45   10.882378   10.318084  0.819426     0.808660
51   10.882678   10.377565  0.818743     0.807937
53   10.883511   10.318192  0.819658     0.808906
41   10.883615   10.321937  0.819411     0.808644
49   10.885020   10.323704  0.819634     0.808881
67   10.891583   10.375288  0.818624     0.807811
71   10.892798   10.371032  0.818640     0.807827
69   10.893877   10.319308  0.819533     0.808774
3    10.894359   10.375289  0.818638     0.807825
65   10.894906   10.323567  0.819515     0.808754
7    10.895314   10.371038  0.818654     0.807842
99   10.895501   10.374548  0.818612     0.807798
5    10.896833   10.319345  0.819549     0.808790
103  10.897006   10.370225  0.818629     0.807815
101  10.897292   10.318764  0.819523     0.808763
1    10.897854   10.323600  0.819530     0.808770
35   10.898273   10.374561  0.818626     0.807813
97   10.898478   10.323044  0.819504     0.808743
39   10.899511   10.370243  0.818643     0.807830
37   10.900251   10.318811  0.819538     0.808779
33   10.901430   10.323086  0.819520     0.808759

[128 rows x 4 columns]

In [14]:
### For Calculating Contributions
def getRawContributions(XTrainScaledConst,ypred_exp,betaCoef,column_names):
    raw_contibutions = np.zeros(XTrainScaledConst.shape)
    for i in range(0,len(ypred_exp)):
        for j in range(0,len(column_names)):
            if(j!=0):
                contribution = ypred_exp[i]-(ypred_exp[i]/exp(betaCoef[j]*XTrainScaledConst.values[i][j]))
            else:
                contribution = ypred_exp[i]-(ypred_exp[i]/exp(betaCoef[j]))
            raw_contibutions[i][j] = contribution
    return raw_contibutions

def getScaledContributions(XTrainScaledConst,ypred_exp,raw_contibutions,column_names):
    scaled_contribution = np.zeros(XTrainScaledConst.shape)
    for i in range(0,len(ypred_exp)):
        cont_factor = np.sum(np.absolute(raw_contibutions[i]))
        y_sum_cont = np.sum(raw_contibutions[i])
        y_actual = ypred_exp[i]
        for j in range(0,len(column_names)):
            scaled_contribution[i][j] = raw_contibutions[i][j] + ((abs(raw_contibutions[i][j])*(y_actual-y_sum_cont))/cont_factor)
    return scaled_contribution

def getPercentMarketingContribution(scaled_contribution_df,marketing_cols):
    total = scaled_contribution_df.sum()
    total = total.reset_index()
    total.columns= ['variables','value']
    #     'ln_ads_POSRebate_D1_S5'
    #marketing_cols= ['ln_ads_TVSpend_D1_S8','ln_ads_Facebook_D3_S5','ln_ads_Video_D3_S4','ln_ads_Display_D3_S5','ln_ads_Search_D3_S4','ln_POSRebate','ln_TradeDisplayPlacement']
    def add_marketing_variable(row):
        if(row['variables'] in marketing_cols):
            return "marketing"
        else:
            return "others"

    total["Marketing_Variables"] = total.apply(lambda row:add_marketing_variable(row),axis=1)
    total = total[total["variables"]!="Title"]
    grouped = total.groupby('Marketing_Variables')['value'].sum().reset_index()
    sum_value = grouped['value'].sum()
    grouped['percent'] = (grouped['value']/sum_value)*100
    total["Percent Marketing Contribution"] = (total["value"] / total["value"].sum())*100
    return (grouped,total)

### For VIF Analysis
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant
def variance_inflation_factors(exog_df):
    '''
    Parameters
    ----------
    exog_df : dataframe, (nobs, k_vars)
        design matrix with all explanatory variables, as for example used in
        regression.

    Returns
    -------
    vif : Series
        variance inflation factors
    '''
    exog_df = add_constant(exog_df)
    vifs = pd.Series(
        [1 / (1. - OLS(exog_df[col].values, 
                       exog_df.loc[:, exog_df.columns != col].values).fit().rsquared) 
         for col in exog_df],
        index=exog_df.columns,
        name='VIF'
    )
    return vifs



In [15]:
df_ads['ln_ASP']

0      0.000000
1      0.000000
2      0.000000
3      0.000000
4      0.000000
5      0.000000
6      0.000000
7      0.000000
8      0.000000
9      0.000000
10     0.000000
11     0.000000
12     0.000000
13     0.000000
14     0.000000
15     0.000000
16     0.000000
17     0.000000
18     0.000000
19     0.000000
20     0.000000
21     0.000000
22     0.000000
23     0.000000
24     0.000000
25     0.000000
26     0.000000
27     0.000000
28     0.000000
29     0.000000
         ...   
575    3.423334
576    3.418898
577    3.245458
578    3.243366
579    3.238010
580    3.160335
581    3.114130
582    3.123141
583    3.135852
584    3.159378
585    3.145334
586    3.081471
587    3.076643
588    2.919185
589    3.008848
590    3.003938
591    3.076717
592    3.049744
593    3.031298
594    3.056021
595    3.137966
596    3.134243
597    3.142191
598    3.171795
599    3.079841
600    3.087734
601    3.089761
602    3.106006
603    3.081716
604    3.089770
Name: ln_ASP, dtype: flo

### Model by removing Pre Order from AD

In [15]:
## Model params of presentation 

# nonMarketingCols = ['ln_ASP', 'Dec', 'Jun', 'Calender Week', '1st week', '2nd week', 'Is Action',
#                     'Is Open World', 'same genre', 'different genre', 'Christmas Flag', 'Thanksgiving Flag', 
#                     'Week Number']

# nonMarketingCols = ['Dec', 'Mar', 'Jun', 'Calender Week', '1st week', '2nd week', 'Is Action', 'Is Open World',\
#                     'different genre', 'Thanksgiving Flag', 'Week Number', 'TubularViews',]

# nonMarketingCols = ['Dec','Mar' ,'Jun', 'Calender Week', '1st week', '2nd week', 'Is Action', \
#                     'Is Open World', 'different genre','Week Number','Thanksgiving Flag', 'TubularViews',\
#                    'Facebook_1st week']

nonMarketingCols = ['Calender Week', 'Dec', 'Mar', 'Jul', '1st week', '2nd week', 'Is Action', 'Is Open World',\
                    'same genre', 'different genre', 'Week Number', 'TubularViews']

marketingLnCols = ['ln_ads_TVSpend_D1_S8','ln_ads_Facebook_D13_S7','ln_ads_Video_D3_S4','ln_ads_Display_D13_S15',\
                   'ln_ads_Search_D3_S4','ln_POSRebate','ln_TradeDisplayPlacement']
# 'ln_ads_POSRebate_D3_S3','ln_ads_TradeDisplayPlacement_D1_S5'

marketing_non_ads_cols = ['TVSpend','Facebook','Video','Display','Search','POSRebate','TradeDisplayPlacement']
colsForModel = nonMarketingCols
colsForModel.extend(marketingLnCols)
df_overall = df_ads.copy()
df_overall = df_overall[(df_overall['Week Number']>=0)]
df_overall = df_overall.drop(['Install base'],axis=1)
print(colsForModel)
#---------------------------#
testGames= ['Lego Ninjago Movie Videogame']
model_dict = calculateLinearRegression(df_overall,colsForModel,nonMarketingCols,marketingLnCols,testGames)
lr_model_ols = model_dict["lr_model_ols"]
XTrainScaledConst = model_dict["XTrainScaledConst"]
XTestScaledConst = model_dict["XTestScaledConst"]
Y_train = model_dict["Y_train"]
Y_test = model_dict["Y_test"]
X_train_all = model_dict["X_train_all"]
X_test_all = model_dict["X_test_all"]
column_names = model_dict["column_names"]
ypredict_train = lr_model_ols.predict(XTrainScaledConst)
ypredict_test = lr_model_ols.predict(XTestScaledConst)
train_mape_ln = mean_absolute_percentage_error(Y_train,ypredict_train.values)
test_mape_ln = mean_absolute_percentage_error(Y_test,ypredict_test.values)
train_mape = mean_absolute_percentage_error(np.exp(Y_train.values),np.exp(ypredict_train.values))
test_mape = mean_absolute_percentage_error(np.exp(Y_test.values),np.exp(ypredict_test.values))
train_rsquare =  Rsquare(Y_train,ypredict_train.values)
train_adjrsquare = adjRsquare(Y_train,ypredict_train.values,XTrainScaledConst.shape[1]-1)
betaCoef = lr_model_ols.params
ypred = lr_model_ols.predict(XTrainScaledConst)
ypred_exp = np.exp(ypred.values)
y_pred_exp_test = np.exp(ypredict_test.values)

['Calender Week', 'Dec', 'Mar', 'Jul', '1st week', '2nd week', 'Is Action', 'Is Open World', 'same genre', 'different genre', 'Week Number', 'TubularViews', 'ln_ads_TVSpend_D1_S8', 'ln_ads_Facebook_D13_S7', 'ln_ads_Video_D3_S4', 'ln_ads_Display_D13_S15', 'ln_ads_Search_D3_S4', 'ln_POSRebate', 'ln_TradeDisplayPlacement']


C:\Users\Sarthak\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


217    51
569    27
564    22
573    31
460    11
238    20
166     5
154    45
562    20
137    28
233    15
505     4
140    31
478    29
43     27
602     8
593    51
557    15
597     3
470    21
245    27
68     52
603     9
221     3
507     6
70      2
146    37
474    25
234    16
577    35
       ..
473    24
50     34
559    17
151    42
492    43
212    46
497    48
596     2
257    39
500    51
78     10
401    45
570    28
491    42
232    14
84     16
555    13
46     30
490    41
155    46
40     24
148    39
418    10
82     14
152    43
63     47
223     5
411     3
147    38
65     49
Name: Calender Week, dtype: int64

In [16]:
y_pred_exp_test

array([ 2791.65047835, 13970.79333232,  2902.60118531,  2964.69971596,
        7372.64364529,  2938.28451257, 22387.39378808, 29967.56068808,
        3441.91176989,  3120.89205254,  3087.97188502,  2953.69540301,
       29131.32047523,  3040.41482846,  8645.77718192, 13723.23375396,
        2851.93527596,  8446.81303519,  4110.74016554, 13886.09680359,
        3073.55233345,  4319.51721499,  3491.81450848, 21985.48477582,
        4278.19321123,  4187.43088383,  3269.82703763, 13890.95684376,
        3223.85749649, 13344.49737511,  3202.80348286,  3050.53226907,
        3485.06825057,  3255.01883821,  2803.5633585 ,  4195.45570151,
        2895.98113529,  2757.51192409,  3296.04093665, 13902.65339235,
        2907.45925768, 17951.80792883, 12323.72050205,  2794.08779456,
        2544.50586986,  3394.80956182,  2993.58486724, 10158.88125464,
        2675.27189384,  3207.44838844,  2833.83027851,  2770.99108767,
        2087.13970802])

In [18]:
#X_train_all.to_excel(path+"\\X_train_all.xlsx")

In [17]:
lr_model_ols.summary(xname=list(column_names))

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.638
Model:                            OLS   Adj. R-squared:                  0.610
Method:                 Least Squares   F-statistic:                     23.33
Date:                Wed, 14 Aug 2019   Prob (F-statistic):           5.39e-45
Time:                        11:23:57   Log-Likelihood:                -268.05
No. Observations:                 272   AIC:                             576.1
Df Residuals:                     252   BIC:                             648.2
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                        7.8133      0.259     30.219      0.000       7.304       8.323
Calender Week               -0.0025      0.004     -0.694      0.488      -0.010       0.005
Dec                          0.5859      0.179      3.270      0.001       0.233       0.939
Mar                          0.3004      0.157      1.919      0.056      -0.008       0.609
Jul                          0.1518      0.157      0.964      0.336      -0.158       0.462
1st week                     0.2279      0.422      0.540      0.589      -0.603       1.059
2nd week                    -0.4554      0.440     -1.034      0.302      -1.323       0.412
Is Action                    0.1677      0.113      1.482      0.140      -0.055       0.391
Is Open World                0.4571      0.135      3.375      0.001       0.190       0.724
same genre                  -0.3496      0.423     -0.826      0.410      -1.183       0.484
different genre             -0.0443      0.109     -0.409      0.683      -0.258       0.169
Week Number                  0.0004      0.006      0.075      0.940      -0.011       0.012
TubularViews              1.288e-07   3.41e-08      3.777      0.000    6.16e-08    1.96e-07
ln_ads_TVSpend_D1_S8         0.0034      0.015      0.223      0.824      -0.027       0.034
ln_ads_Facebook_D13_S7       0.0251      0.019      1.321      0.188      -0.012       0.063
ln_ads_Video_D3_S4           0.0301      0.021      1.407      0.161      -0.012       0.072
ln_ads_Display_D13_S15       0.0381      0.016      2.344      0.020       0.006       0.070
ln_ads_Search_D3_S4          0.0049      0.022      0.220      0.826      -0.039       0.048
ln_POSRebate                 0.0062      0.012      0.530      0.597      -0.017       0.029
ln_TradeDisplayPlacement     0.0875      0.014      6.045      0.000       0.059       0.116
==============================================================================
Omnibus:                        5.636   Durbin-Watson:                   1.861
Prob(Omnibus):                  0.060   Jarque-Bera (JB):                5.330
Skew:                           0.317   Prob(JB):                       0.0696
Kurtosis:                       3.262   Cond. No.                     3.26e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.26e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [18]:
## Vif
df_vif = XTrainScaledConst
df_vif = df_vif.drop(['const'],axis=1)
vif = variance_inflation_factors(df_vif)
vif = vif.reset_index()
vif = vif.sort_values(by=['VIF'],ascending=0)
vif

index        VIF
0                      const  40.089987
15        ln_ads_Video_D3_S4   6.127836
13      ln_ads_TVSpend_D1_S8   4.989385
11               Week Number   4.595686
17       ln_ads_Search_D3_S4   4.037850
12              TubularViews   3.191721
14    ln_ads_Facebook_D13_S7   2.962945
19  ln_TradeDisplayPlacement   2.570432
16    ln_ads_Display_D13_S15   2.254891
1              Calender Week   1.974271
18              ln_POSRebate   1.959773
2                        Dec   1.888742
6                   2nd week   1.408766
8              Is Open World   1.350492
5                   1st week   1.293324
3                        Mar   1.226387
7                  Is Action   1.204849
9                 same genre   1.172043
10           different genre   1.130028
4                        Jul   1.105367

In [19]:
df_predicted_train_original = X_train_all.copy()
df_predicted_test_original = X_test_all.copy()

df_predicted_train_original['ln_Units'] = Y_train.values
df_predicted_train_original['ln_Units-Predicted'] = ypredict_train.values
df_predicted_train_original['Units'] = np.exp(Y_train.values)
df_predicted_train_original['Units-Predicted'] = np.exp(ypredict_train.values)

df_predicted_test_original['ln_Units'] = Y_test.values
df_predicted_test_original['ln_Units-Predicted'] = ypredict_test.values
df_predicted_test_original['Units'] = np.exp(Y_test.values)
df_predicted_test_original['Units-Predicted'] = np.exp(ypredict_test.values)

#df_predicted_train_original.to_excel(path+"\\capped\\only_units\\prediction_train_case1.xlsx")
#df_predicted_test_original.to_excel(path+"\\capped\\only_units\\predicted_test_case1.xlsx")

In [20]:
print(w_mape(np.exp(Y_train.values),np.exp(ypredict_train.values)))
print(w_mape(np.exp(Y_test.values),np.exp(ypredict_test.values)))

50.56353616483686
55.59988898561595


In [21]:
def isBothContributionPositive(model_dict,marketingLnCols):
    lr_model_ols = model_dict["lr_model_ols"]
    XTrainScaledConst = model_dict["XTrainScaledConst"]
    XTestScaledConst = model_dict["XTestScaledConst"]
    Y_train = model_dict["Y_train"]
    Y_test = model_dict["Y_test"]
    column_names = model_dict["column_names"]
    ypredict_train = lr_model_ols.predict(XTrainScaledConst)
    ypredict_test = lr_model_ols.predict(XTestScaledConst)
    train_mape_ln = mean_absolute_percentage_error(Y_train,ypredict_train.values)
    test_mape_ln = mean_absolute_percentage_error(Y_test,ypredict_test.values)
    train_mape = mean_absolute_percentage_error(np.exp(Y_train.values),np.exp(ypredict_train.values))
    test_mape = mean_absolute_percentage_error(np.exp(Y_test.values),np.exp(ypredict_test.values))
    train_rsquare =  Rsquare(Y_train,ypredict_train.values)
    train_adjrsquare = adjRsquare(Y_train,ypredict_train.values,XTrainScaledConst.shape[1]-1)
    betaCoef = lr_model_ols.params
    ypred = lr_model_ols.predict(XTrainScaledConst)
    ypred_exp = np.exp(ypred.values)
    raw_contribution = getRawContributions(XTrainScaledConst,ypred_exp,betaCoef,column_names)
    scaled_contribution = getScaledContributions(XTrainScaledConst,ypred_exp,raw_contribution,column_names)
    scaled_contribution_df = pd.DataFrame(scaled_contribution,columns = column_names)
    grouped,total = getPercentMarketingContribution(scaled_contribution_df,marketingLnCols)
    total["Percent Marketing Contribution"] = (total["value"] / total["value"].sum())*100
    others_percent = grouped.loc[1,"percent"]
    marketing_percent = grouped.loc[0,"percent"]
    return (marketing_percent,others_percent,train_mape,test_mape,train_mape_ln,test_mape_ln)


In [22]:
isBothContributionPositive(model_dict,marketingLnCols)

(39.85174293117491,
 60.148257068825096,
 57.2598250383563,
 35.90445417166541,
 5.924510552331739,
 4.164222879966913)

In [23]:
print(train_mape)
print(test_mape)
print(train_mape_ln)
print(test_mape_ln)

57.2598250383563
35.90445417166541
5.924510552331739
4.164222879966913


# ROI Calculation

In [24]:
def getContributionsDF(X,ypred_exp,betaCoef,column_names,X_all):
    raw_contribution = getRawContributions(X,ypred_exp,betaCoef,column_names)
    scaled_contribution = getScaledContributions(X,ypred_exp,raw_contribution,column_names)
    scaled_contribution_df = pd.DataFrame(scaled_contribution,columns = column_names)
    scaled_contribution_df = scaled_contribution_df.set_index(X.index)
    scaled_contribution_df = pd.concat([scaled_contribution_df,X_all[['Title','Week Ending']]],axis=1)
    return scaled_contribution_df

def getROIChannelWise(X,Y,marketingLnCols,marketing_non_ads_cols,total):
    X_df = X.copy()
    Y_df = Y.copy().to_frame()
    Y_df['Units'] = np.exp(Y.values)
    df_train = pd.concat([X_df,Y_df],axis=1)
    #marketing_non_ads_cols = [x.split("_")[2] for x in marketingLnCols] 
    marketing_ln_ads_cols = marketingLnCols
    total_units = df_train['Units'].sum()
    Total_Revenue = (df_train['Total Revenue']).sum()
    marketing_spend = df_train[marketing_non_ads_cols].sum().to_frame(name= "Spend").reset_index()
    marketing_spend['marketing_variables'] = marketing_spend['index']
    marketing_spend = marketing_spend.drop(['index'],axis=1)
    marketing_contribution = total.copy()
    marketing_contribution = marketing_contribution[marketing_contribution.variables.isin(marketing_ln_ads_cols)]
    marketing_contribution['Total Units'] = total_units
    marketing_contribution['Total_Revenue'] = Total_Revenue
    marketing_contribution = marketing_contribution.drop(['value','Marketing_Variables'],axis=1).reset_index()
    marketing = pd.concat([marketing_contribution,marketing_spend],axis=1).drop(['index'],axis=1)
    marketing['ROMI_In_Revenue_Terms'] = (marketing['Total_Revenue'] * marketing['Percent Marketing Contribution'])/(100 * marketing['Spend'])
    marketing['ROMI_In_Units_Terms'] = (marketing['Total Units'] * marketing['Percent Marketing Contribution'])/(100 * marketing['Spend'])
    return marketing

def getROI_title_channel_wise(titles,scaled_contribution_df,X_all,Y,marketingLnCols,marketing_non_ads_cols):
    dic_mkt_tit_chan_wise = {}
    for title in titles:
        scaled_contribution_title = scaled_contribution_df[scaled_contribution_df['Title']==title]
        grouped_title,total_title = getPercentMarketingContribution(scaled_contribution_title,marketingLnCols)
        X_title = X_all[X_all['Title']==title]
        Y_title = Y[Y.index.isin(X_title.index)]
        marketing_title_channel_wise = getROIChannelWise(X_title,Y_title,marketingLnCols,marketing_non_ads_cols,total_title)
        dic_mkt_tit_chan_wise[title] = (marketing_title_channel_wise,grouped_title,total_title)
    return dic_mkt_tit_chan_wise


In [25]:
marketingLnCols

['ln_ads_TVSpend_D1_S8',
 'ln_ads_Facebook_D13_S7',
 'ln_ads_Video_D3_S4',
 'ln_ads_Display_D13_S15',
 'ln_ads_Search_D3_S4',
 'ln_POSRebate',
 'ln_TradeDisplayPlacement']

# ROI  Overall - Channel Wise

In [26]:
scaled_contribution_df = getContributionsDF(XTrainScaledConst.copy(),ypred_exp,betaCoef,column_names,X_train_all.copy())
grouped,total = getPercentMarketingContribution(scaled_contribution_df,marketingLnCols)
marketing_channel_wise = getROIChannelWise(X_train_all,Y_train,marketingLnCols,marketing_non_ads_cols,total)

In [27]:
grouped

Marketing_Variables         value    percent
0           marketing  1.195843e+06  39.851743
1              others  1.804887e+06  60.148257

In [28]:
total

variables         value Marketing_Variables  \
0                      const  1.482717e+06              others   
1              Calender Week -4.031831e+05              others   
2                        Dec  1.457356e+05              others   
3                        Mar  3.476707e+04              others   
4                        Jul  1.759349e+04              others   
5                   1st week  1.757093e+04              others   
6                   2nd week -1.264859e+05              others   
7                  Is Action  1.963903e+05              others   
8              Is Open World  8.867336e+04              others   
9                 same genre -8.665804e+03              others   
10           different genre -3.738665e+04              others   
11               Week Number  1.106004e+04              others   
12              TubularViews  3.861009e+05              others   
13      ln_ads_TVSpend_D1_S8  3.915801e+04           marketing   
14    ln_ads_Facebook_D13_S7  1.015117e+05           marketing   
15        ln_ads_Video_D3_S4  2.435946e+05           marketing   
16    ln_ads_Display_D13_S15  1.698963e+05           marketing   
17       ln_ads_Search_D3_S4  2.809352e+04           marketing   
18              ln_POSRebate  4.899335e+04           marketing   
19  ln_TradeDisplayPlacement  5.645960e+05           marketing   

    Percent Marketing Contribution  
0                        49.411863  
1                       -13.436164  
2                         4.856669  
3                         1.158620  
4                         0.586307  
5                         0.585555  
6                        -4.215169  
7                         6.544750  
8                         2.955059  
9                        -0.288790  
10                       -1.245918  
11                        0.368578  
12                       12.866897  
13                        1.304949  
14                        3.382901  
15                        8.117842  
16                        5.661830  
17                        0.936223  
18                        1.632714  
19                       18.815284

In [29]:
marketing_channel_wise

variables  Percent Marketing Contribution  Total Units  \
0      ln_ads_TVSpend_D1_S8                        1.304949    3477740.0   
1    ln_ads_Facebook_D13_S7                        3.382901    3477740.0   
2        ln_ads_Video_D3_S4                        8.117842    3477740.0   
3    ln_ads_Display_D13_S15                        5.661830    3477740.0   
4       ln_ads_Search_D3_S4                        0.936223    3477740.0   
5              ln_POSRebate                        1.632714    3477740.0   
6  ln_TradeDisplayPlacement                       18.815284    3477740.0   

   Total_Revenue         Spend    marketing_variables  ROMI_In_Revenue_Terms  \
0   1.363231e+08  8.857741e+06                TVSpend               0.200835   
1   1.363231e+08  5.971686e+05               Facebook               7.722571   
2   1.363231e+08  1.853714e+06                  Video               5.969906   
3   1.363231e+08  1.006479e+06                Display               7.668703   
4   1.363231e+08  2.640967e+05                 Search               4.832655   
5   1.363231e+08  1.473283e+07              POSRebate               0.151075   
6   1.363231e+08  3.066379e+06  TradeDisplayPlacement               8.364780   

   ROMI_In_Units_Terms  
0             0.005124  
1             0.197011  
2             0.152298  
3             0.195636  
4             0.123286  
5             0.003854  
6             0.213394

In [56]:
marketing_channel_wise[marketing_channel_wise.marketing_variables=="Display"]['ROMI_In_Revenue_Terms'].values[0]

7.668702969270878

# ROI  Train Title- Channel Wise 

In [41]:
## In the dictionary of roi - title-channel wise, we get roi and grouped contributions
train_titles = ['Cars-3-Driven-to-Win','Lego Incredibles','Lego Marvel Super Hero 2','Lego Super DC','Lego Worlds',
                'Scribblenauts Showdown/Scribblenauts Fiesta']
scaled_contribution_df_train = getContributionsDF(XTrainScaledConst.copy(),ypred_exp,betaCoef,column_names,X_train_all)
dic_mkt_tit_chan_wise_train = getROI_title_channel_wise(train_titles,scaled_contribution_df_train,X_train_all.copy(),Y_train,marketingLnCols,marketing_non_ads_cols)


In [53]:
X_train_all['ypred'] = ypred_exp
title_pred = X_train_all[['Title','Week Number','Total Units','ypred']]
title_pred.to_excel("D://Affine_Projects//WB_Warner_Bros//MMM//Optimisation//22072019//pred.xlsx")
title_pred.groupby(['Title'])['ypred'].sum()

Title
Cars-3-Driven-to-Win                           355509.914414
Lego Incredibles                               508125.936404
Lego Marvel Super Hero 2                       848875.037727
Lego Super DC                                  452199.362516
Lego Worlds                                    581588.607647
Scribblenauts Showdown/Scribblenauts Fiesta    254431.826055
Name: ypred, dtype: float64

In [36]:
### Scaled Contribution for each title
dic_mkt_tit_chan_wise_train['Lego Super DC'][2]

variables          value Marketing_Variables  \
0                      const  196630.517847              others   
1              Calender Week  -75349.378822              others   
2                        Dec   28684.138513              others   
3                        Mar    2867.596738              others   
4                        Jul       0.000000              others   
5                   1st week    3809.398053              others   
6                   2nd week  -31627.181280              others   
7                  Is Action   30371.157782              others   
8              Is Open World       0.000000              others   
9                 same genre       0.000000              others   
10           different genre   -4147.010265              others   
11               Week Number     578.677770              others   
12              TubularViews   83788.817381              others   
13      ln_ads_TVSpend_D1_S8    8704.114405           marketing   
14    ln_ads_Facebook_D13_S7   36451.355771           marketing   
15        ln_ads_Video_D3_S4   46651.478706           marketing   
16    ln_ads_Display_D13_S15       0.000000           marketing   
17       ln_ads_Search_D3_S4    8857.296554           marketing   
18              ln_POSRebate    9026.078489           marketing   
19  ln_TradeDisplayPlacement  106902.304873           marketing   

    Percent Marketing Contribution  
0                        43.483148  
1                       -16.662867  
2                         6.343251  
3                         0.634144  
4                         0.000000  
5                         0.842416  
6                        -6.994079  
7                         6.716320  
8                         0.000000  
9                         0.000000  
10                       -0.917076  
11                        0.127970  
12                       18.529176  
13                        1.924840  
14                        8.060904  
15                       10.316573  
16                        0.000000  
17                        1.958715  
18                        1.996040  
19                       23.640525

In [37]:
dic_mkt_tit_chan_wise_train['Lego Super DC'][0]

variables  Percent Marketing Contribution  Total Units  \
0      ln_ads_TVSpend_D1_S8                        1.924840     456433.0   
1    ln_ads_Facebook_D13_S7                        8.060904     456433.0   
2        ln_ads_Video_D3_S4                       10.316573     456433.0   
3    ln_ads_Display_D13_S15                        0.000000     456433.0   
4       ln_ads_Search_D3_S4                        1.958715     456433.0   
5              ln_POSRebate                        1.996040     456433.0   
6  ln_TradeDisplayPlacement                       23.640525     456433.0   

   Total_Revenue         Spend    marketing_variables  ROMI_In_Revenue_Terms  \
0   2.070915e+07  1.367641e+06                TVSpend               0.291464   
1   2.070915e+07  2.022515e+05               Facebook               8.253807   
2   2.070915e+07  3.542865e+05                  Video               6.030357   
3   2.070915e+07  0.000000e+00                Display                    NaN   
4   2.070915e+07  1.236983e+05                 Search               3.279213   
5   2.070915e+07  4.322272e+06              POSRebate               0.095636   
6   2.070915e+07  6.964376e+05  TradeDisplayPlacement               7.029706   

   ROMI_In_Units_Terms  
0             0.006424  
1             0.181915  
2             0.132910  
3                  NaN  
4             0.072274  
5             0.002108  
6             0.154936

In [91]:
dic_mkt_tit_chan_wise_train['Scribblenauts Showdown/Scribblenauts Fiesta'][1]

Marketing_Variables          value    percent
0           marketing   51671.311547  20.308509
1              others  202760.514508  79.691491

# ROI Test Title - Channel Wise.

In [92]:
## For Test Title-Channel Wise
test_titles = ['Lego Ninjago Movie Videogame']
scaled_contribution_df_test = getContributionsDF(XTestScaledConst.copy(),y_pred_exp_test,betaCoef,column_names,X_test_all)
dic_mkt_tit_chan_wise_test = getROI_title_channel_wise(test_titles,scaled_contribution_df_test,X_test_all.copy(),Y_test,marketingLnCols,marketing_non_ads_cols)


In [93]:
dic_mkt_tit_chan_wise_test['Lego Ninjago Movie Videogame'][2]

variables          value Marketing_Variables  \
0                      const  219065.732464              others   
1              Calender Week  -50032.468381              others   
2                        Dec   15709.856623              others   
3                        Mar    3789.952713              others   
4                        Jul    1537.631316              others   
5                   1st week    2009.603384              others   
6                   2nd week  -14196.272162              others   
7                  Is Action   33836.456305              others   
8              Is Open World       0.000000              others   
9                 same genre    -740.452737              others   
10           different genre   -3959.884090              others   
11               Week Number    1780.570435              others   
12              TubularViews   13626.481961              others   
13      ln_ads_TVSpend_D1_S8    4606.089436           marketing   
14    ln_ads_Facebook_D13_S7   16714.525187           marketing   
15        ln_ads_Video_D3_S4   22704.929208           marketing   
16    ln_ads_Display_D13_S15   16102.367498           marketing   
17       ln_ads_Search_D3_S4    2660.407980           marketing   
18              ln_POSRebate    8828.313577           marketing   
19  ln_TradeDisplayPlacement   74245.888622           marketing   

    Percent Marketing Contribution  
0                        59.481901  
1                       -13.585084  
2                         4.265624  
3                         1.029068  
4                         0.417506  
5                         0.545658  
6                        -3.854648  
7                         9.187456  
8                         0.000000  
9                        -0.201052  
10                       -1.075209  
11                        0.483470  
12                        3.699935  
13                        1.250670  
14                        4.538417  
15                        6.164964  
16                        4.372201  
17                        0.722368  
18                        2.397111  
19                       20.159641

In [94]:
dic_mkt_tit_chan_wise_test['Lego Ninjago Movie Videogame'][0]

variables  Percent Marketing Contribution  Total Units  \
0      ln_ads_TVSpend_D1_S8                        1.250670     448459.0   
1    ln_ads_Facebook_D13_S7                        4.538417     448459.0   
2        ln_ads_Video_D3_S4                        6.164964     448459.0   
3    ln_ads_Display_D13_S15                        4.372201     448459.0   
4       ln_ads_Search_D3_S4                        0.722368     448459.0   
5              ln_POSRebate                        2.397111     448459.0   
6  ln_TradeDisplayPlacement                       20.159641     448459.0   

   Total_Revenue         Spend    marketing_variables  ROMI_In_Revenue_Terms  \
0   1.746701e+07  1.533098e+05                TVSpend               1.424923   
1   1.746701e+07  2.002589e+05               Facebook               3.958506   
2   1.746701e+07  3.338337e+05                  Video               3.225663   
3   1.746701e+07  3.810707e+04                Display              20.040713   
4   1.746701e+07  1.468425e+04                 Search               8.592618   
5   1.746701e+07  4.861130e+06              POSRebate               0.086133   
6   1.746701e+07  1.014932e+06  TradeDisplayPlacement               3.469480   

   ROMI_In_Units_Terms  
0             0.036584  
1             0.101633  
2             0.082818  
3             0.514538  
4             0.220612  
5             0.002211  
6             0.089078

In [95]:
dic_mkt_tit_chan_wise_test['Lego Ninjago Movie Videogame'][1]

Marketing_Variables          value    percent
0           marketing  145862.521509  39.605373
1              others  222427.207831  60.394627

In [32]:
## Not using this functions. Will have to delete in the future.Have writtten new functions to calculate ROI at overall-channel 
## and Title-Channel Wise.  

# def getROMI(row,numerator,contribution,denominator):
#     if(row[denominator]==0):
#         return 0
#     else:
#         return (row[numerator]*row[contribution])/(100*row[denominator])

# def getROI(X,Y,marketingLnCols,total):
#     X_df = X.copy()
#     Y_df = Y.copy().to_frame()
#     Y_df['Units'] = np.exp(Y.values)
#     df_train = pd.concat([X_df,Y_df],axis=1)
#     marketing_non_ads_cols = [x.split("_")[2] for x in marketingLnCols] 
#     marketing_ln_ads_cols = marketingLnCols
#     total_units = df_train['Units'].sum()
#     title_total_units = df_train.groupby(['Title'])['Units'].sum().to_frame().reset_index()
#     Total_Revenue = (df_train['Total Revenue']).sum()
#     title_total_revenue = df_train.groupby(['Title'])['Total Revenue'].sum().to_frame().reset_index()
#     marketing_spend = df_train[marketing_non_ads_cols].sum().to_frame(name= "Spend").reset_index()
#     title_marketing_spend = df_train.groupby(['Title'])[marketing_non_ads_cols].sum().reset_index()
#     marketing_spend['marketing_variables'] = marketing_spend['index']
#     marketing_spend = marketing_spend.drop(['index'],axis=1)
#     marketing_contribution = total.copy()
#     marketing_contribution = marketing_contribution[marketing_contribution.variables.isin(marketing_ln_ads_cols)]
#     title_marketing_contribution = marketing_contribution.copy()
#     marketing_contribution['Total Units'] = total_units
#     marketing_contribution['Total_Revenue'] = Total_Revenue
#     marketing_contribution = marketing_contribution.drop(['value','Marketing_Variables'],axis=1).reset_index()
#     title_marketing_contribution = title_marketing_contribution.drop(['value','Marketing_Variables'],axis=1).reset_index()
#     title_marketing_contribution = title_marketing_contribution[['variables','Percent Marketing Contribution']]
#     title_marketing_contribution['variables'] = title_marketing_contribution.apply(lambda row:row['variables'].split("_")[2],axis=1)
#     title_marketing_contribution.set_index('variables',inplace=True)
#     title_marketing_spend = title_marketing_spend.merge(title_total_units,how='inner',on='Title')
#     title_marketing_spend  = title_marketing_spend.merge(title_total_revenue,how='inner',on='Title')
#     title_marketing_spend['Contribution_TVSpend'] = title_marketing_contribution.loc['TVSpend','Percent Marketing Contribution']
#     title_marketing_spend['Contribution_Facebook'] = title_marketing_contribution.loc['Facebook','Percent Marketing Contribution']
#     title_marketing_spend['Contribution_Video'] = title_marketing_contribution.loc['Video','Percent Marketing Contribution']
#     title_marketing_spend['Contribution_Display'] = title_marketing_contribution.loc['Display','Percent Marketing Contribution']
#     title_marketing_spend['Contribution_Search'] = title_marketing_contribution.loc['Search','Percent Marketing Contribution']
#     title_marketing_spend['Contribution_POSRebate'] = title_marketing_contribution.loc['POSRebate','Percent Marketing Contribution']
#     title_marketing_spend['Contribution_TradeDisplayPlacement'] = title_marketing_contribution.loc['TradeDisplayPlacement','Percent Marketing Contribution']
#     title_marketing =  title_marketing_spend.copy()
#     marketing = pd.concat([marketing_contribution,marketing_spend],axis=1).drop(['index'],axis=1)
#     marketing['ROMI_In_Revenue_Terms'] = (marketing['Total_Revenue'] * marketing['Percent Marketing Contribution'])/(100 * marketing['Spend'])
#     marketing['ROMI_In_Units_Terms'] = (marketing['Total Units'] * marketing['Percent Marketing Contribution'])/(100 * marketing['Spend'])
#     for col in marketing_non_ads_cols:
#         title_marketing['ROMI_Revenue_'+col] = title_marketing.apply(lambda row:getROMI(row,'Total Revenue','Contribution_'+col,col),axis=1)
#         title_marketing['ROMI_Units_'+col] = title_marketing.apply(lambda row:getROMI(row,'Units','Contribution_'+col,col),axis=1)
#     return (marketing,title_marketing.pivot_table(columns='Title'))


In [133]:
# marketing_train,title_marketing_train = getROI(X_train_all,Y_train,marketingLnCols,total)
# marketing_test,title_marketing_test = getROI(X_test_all,Y_test,marketingLnCols,total)

In [134]:
# marketing_train

In [44]:
##title_marketing_combined = pd.concat([title_marketing_train,title_marketing_test],axis=1)

In [135]:
#### ROI Calculation
# X_train_df = X_train_all.copy()
# Y_train_df = Y_train.copy().to_frame()
# Y_train_df['Units'] = np.exp(Y_train.values)
# df_train = pd.concat([X_train_df,Y_train_df],axis=1)
# marketing_non_ads_cols = [x.split("_")[2] for x in marketingLnCols] 
# marketing_ln_ads_cols = marketingLnCols 
# #df_train.to_excel(path+"\\capped\\only_units\\raw_data_roi.xlsx")
# print(marketing_non_ads_cols)
# df_train

In [136]:
### ROI

# total_units = df_train['Units'].sum()
# title_total_units = df_train.groupby(['Title'])['Units'].sum().to_frame().reset_index()
# Total_Revenue = (df_train['Total Revenue']).sum()
# title_total_revenue = df_train.groupby(['Title'])['Total Revenue'].sum().to_frame().reset_index()
# marketing_spend = df_train[marketing_non_ads_cols].sum().to_frame(name= "Spend").reset_index()
# title_marketing_spend = df_train.groupby(['Title'])[marketing_non_ads_cols].sum().reset_index()
# marketing_spend['marketing_variables'] = marketing_spend['index']
# marketing_spend = marketing_spend.drop(['index'],axis=1)
# marketing_contribution = total.copy()
# marketing_contribution = marketing_contribution[marketing_contribution.variables.isin(marketing_ln_ads_cols)]
# title_marketing_contribution = marketing_contribution.copy()
# marketing_contribution['Total Units'] = total_units
# marketing_contribution['Total_Revenue'] = Total_Revenue
# marketing_contribution = marketing_contribution.drop(['value','Marketing_Variables'],axis=1).reset_index()
# title_marketing_contribution = title_marketing_contribution.drop(['value','Marketing_Variables'],axis=1).reset_index()
# title_marketing_contribution = title_marketing_contribution[['variables','Percent Marketing Contribution']]

In [137]:
# title_marketing_contribution['variables'] = title_marketing_contribution.apply(lambda row:row['variables'].split("_")[2],axis=1)
# title_marketing_contribution.set_index('variables',inplace=True)
# title_marketing_contribution

In [138]:
# title_marketing_spend = title_marketing_spend.merge(title_total_units,how='inner',on='Title')
# title_marketing_spend  = title_marketing_spend.merge(title_total_revenue,how='inner',on='Title')
# title_marketing_spend['Contribution_TVSpend'] = title_marketing_contribution.loc['TVSpend','Percent Marketing Contribution']
# title_marketing_spend['Contribution_Facebook'] = title_marketing_contribution.loc['Facebook','Percent Marketing Contribution']
# title_marketing_spend['Contribution_Video'] = title_marketing_contribution.loc['Video','Percent Marketing Contribution']
# title_marketing_spend['Contribution_Display'] = title_marketing_contribution.loc['Display','Percent Marketing Contribution']
# title_marketing_spend['Contribution_Search'] = title_marketing_contribution.loc['Search','Percent Marketing Contribution']
# title_marketing_spend['Contribution_POSRebate'] = title_marketing_contribution.loc['POSRebate','Percent Marketing Contribution']
# title_marketing_spend['Contribution_TradeDisplayPlacement'] = title_marketing_contribution.loc['TradeDisplayPlacement','Percent Marketing Contribution']


In [139]:
# title_marketing =  title_marketing_spend.copy()
# title_marketing 

In [140]:
# marketing = pd.concat([marketing_contribution,marketing_spend],axis=1).drop(['index'],axis=1)
# marketing['ROMI_In_Revenue_Terms'] = (marketing['Total_Revenue'] * marketing['Percent Marketing Contribution'])/(100 * marketing['Spend'])
# marketing['ROMI_In_Units_Terms'] = (marketing['Total Units'] * marketing['Percent Marketing Contribution'])/(100 * marketing['Spend'])

In [141]:
# marketing

In [142]:
# def getROMI(row,numerator,contribution,denominator):
#     if(row[denominator]==0):
#         return 0
#     else:
#         return (row[numerator]*row[contribution])/(100*row[denominator])

# for col in marketing_non_ads_cols:
#     title_marketing['ROMI_Revenue_'+col] = title_marketing.apply(lambda row:getROMI(row,'Total Revenue','Contribution_'+col,col),axis=1)
#     title_marketing['ROMI_Units_'+col] = title_marketing.apply(lambda row:getROMI(row,'Units','Contribution_'+col,col),axis=1)
# title_marketing

In [143]:
# title_marketing.pivot_table(columns='Title').to_excel(path+"\\ROMI_Title-Channel_Case2.xlsx")

In [342]:
###  Create all combinations of variable cols so as to check if we can get any positive beta coef combination

import itertools

# 'Display_1st week'
f_cols =['Calender Week']
variable_cols = ['Is Adventure','Dec','Mar','Jun','Jul','Christmas Flag','Thanksgiving Flag','1st week',\
                 '2nd week','Is Action','Is Open World','same genre','different genre','Sales Trend',
                'Week Number','TubularViews']
all_combinations =[] 
## 12, len(variable_cols)+1
for L in range(11, len(variable_cols)+1):
    for subset in itertools.combinations(variable_cols, L):
        fix = f_cols.copy()
        fix.extend(list(subset))
        all_combinations.append(fix)


In [343]:
#len(variable_cols)
len(all_combinations)

6885

In [310]:
# titles = ['Cars-3-Driven-to-Win','Lego Incredibles','Lego Marvel Super Hero 2','Lego Ninjago Movie Videogame','Lego Super DC','Lego Worlds','Scribblenauts Showdown/Scribblenauts Fiesta']
# titles_combinations = [list(subset) for subset in itertools.combinations(titles,2)]
# titles_combinations

In [311]:
# pos_testGames_comb =[]
# count=1
# for testGames in titles_combinations:
#     print(count)
#     count = count+1
#     positive_comb_arr = [] 
#     non_positive_comb_arr = []
#     for combination in all_combinations:
#         var_df = df_ads1[combination]
#         marketing_original = df_ads1[marketing_original_cols]
#         df_concat_var = pd.concat([var_df,marketing_original,df_marketing_percent_ln],axis=1)
#         model_dict = calculateLinearRegression(df_concat_var,marketing_original_cols,testGames)
#         lr_model_ols = model_dict["lr_model_ols"]
#         column_names = model_dict["column_names"]
#         model_params = lr_model_ols.params
#         marketing_betacoef = model_params[-7:]
#         if((all(i >= 0 for i in marketing_betacoef)) & (model_params[0]>=0)):
#             positive_comb_arr.append(combination)
#         else:
#             non_positive_comb_arr.append(combination)
#     if(len(positive_comb_arr)>0):
#         print("Hurray!!!!!")
#         pos_testGames_comb.append((testGames,positive_comb_arr))



In [344]:
### Select all the combination where we have positive beta coef of marketing variables and intercept and add that combination in the array
positive_comb_arr = [] 
non_positive_comb_arr = []
count=1
testGames= ['Lego Ninjago Movie Videogame']
### 'ln_ads_POSRebate_D3_S3'
marketingLnCols = ['ln_ads_TVSpend_D1_S8','ln_ads_Facebook_D3_S5','ln_ads_Video_D3_S4',\
                   'ln_ads_Display_D11_S15','ln_ads_Search_D3_S4','ln_POSRebate',\
                   'ln_TradeDisplayPlacement']
for combination in all_combinations:
    print(count)
    count = count+1    
    colsForModel = combination
    colsForModel.extend(marketingLnCols)
    model_dict = calculateLinearRegression(df_overall.copy(),colsForModel,nonMarketingCols,marketingLnCols,testGames)
    lr_model_ols = model_dict["lr_model_ols"]
    column_names = model_dict["column_names"]
    model_params = lr_model_ols.params
    marketing_betacoef = model_params[-7:]
    if((all(i >= 0 for i in marketing_betacoef)) & (model_params[0]>=0)):
        positive_comb_arr.append(combination)
    else:
        non_positive_comb_arr.append(combination)
        



1
2
3
4


C:\Users\Sarthak\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\Sarthak\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\Sarthak\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\Sarthak\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


5
6
7
8


C:\Users\Sarthak\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\Sarthak\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\Sarthak\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\Sarthak\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


9
10
11
12


...

6877
6878
6879
6880


C:\Users\Sarthak\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\Sarthak\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\Sarthak\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\Sarthak\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


6881
6882
6883
6884


C:\Users\Sarthak\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\Sarthak\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\Sarthak\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\Sarthak\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


6885


In [345]:
print(len(positive_comb_arr))
positive_comb_arr

107


[['Calender Week',
  'Dec',
  'Mar',
  'Jun',
  'Jul',
  'Christmas Flag',
  'Thanksgiving Flag',
  '1st week',
  'Is Open World',
  'different genre',
  'Sales Trend',
  'TubularViews',
  'ln_ads_TVSpend_D1_S8',
  'ln_ads_Facebook_D3_S5',
  'ln_ads_Video_D3_S4',
  'ln_ads_Display_D11_S15',
  'ln_ads_Search_D3_S4',
  'ln_POSRebate',
  'ln_TradeDisplayPlacement'],
 ['Calender Week',
  'Dec',
  'Mar',
  'Jun',
  'Jul',
  'Christmas Flag',
  'Thanksgiving Flag',
  '2nd week',
  'Is Open World',
  'different genre',
  'Sales Trend',
  'TubularViews',
  'ln_ads_TVSpend_D1_S8',
  'ln_ads_Facebook_D3_S5',
  'ln_ads_Video_D3_S4',
  'ln_ads_Display_D11_S15',
  'ln_ads_Search_D3_S4',
  'ln_POSRebate',
  'ln_TradeDisplayPlacement'],
 ['Calender Week',
  'Dec',
  'Mar',
  'Jun',
  'Jul',
  'Christmas Flag',
  '1st week',
  '2nd week',
  'Is Action',
  'same genre',
  'Sales Trend',
  'TubularViews',
  'ln_ads_TVSpend_D1_S8',
  'ln_ads_Facebook_D3_S5',
  'ln_ads_Video_D3_S4',
  'ln_ads_Display_D11_

In [346]:
#### Select the combination where we have both marketing and others contribution to be positive so as to have desirable contributions.
positive_cont = []
negative_cont = []
count =1
for combination in positive_comb_arr:
    dic = {}
    print(count)
    count = count+1
    #        'ln_ads_POSRebate_D1_S5'
    colsForModel = combination
    nonMarketingCols = [x for x in combination if x not in marketingLnCols]
    model_dict = calculateLinearRegression(df_overall.copy(),colsForModel,nonMarketingCols,marketingLnCols,testGames)
    lr_model_ols = model_dict["lr_model_ols"]
    column_names = model_dict["column_names"]
    model_params = lr_model_ols.params
    marketing_contibution,others_contribution,train_mape,test_mape,train_mape_ln,test_mape_ln = isBothContributionPositive(model_dict,marketingLnCols)
    
    XTrainScaledConst = model_dict["XTrainScaledConst"]
    Y_train = model_dict["Y_train"]
    Y_test = model_dict["Y_test"]
    X_train_all = model_dict["X_train_all"]
    X_test_all = model_dict["X_test_all"]
    column_names = model_dict["column_names"]
    ypredict_train = lr_model_ols.predict(XTrainScaledConst)
    betaCoef = lr_model_ols.params
    ypred = lr_model_ols.predict(XTrainScaledConst)
    ypred_exp = np.exp(ypred.values)
    y_pred_exp_test = np.exp(ypredict_test.values)
    
    scaled_contribution_df = getContributionsDF(XTrainScaledConst.copy(),ypred_exp,betaCoef,column_names,X_train_all.copy())
    grouped,total = getPercentMarketingContribution(scaled_contribution_df,marketingLnCols)
    marketing_channel_wise = getROIChannelWise(X_train_all,Y_train,marketingLnCols,marketing_non_ads_cols,total)
    display_ROI = marketing_channel_wise[marketing_channel_wise.marketing_variables=="Display"]['ROMI_In_Revenue_Terms'].values[0]
    
    if((others_contribution > 0) & (marketing_contibution > 0) & (display_ROI <=10)):
        dic["combination"] = combination
        dic["marketing_contribution"] = marketing_contibution
        dic["others_contribution"] = others_contribution
        dic["train_mape"] = train_mape
        dic["test_mape"] = test_mape
        dic["train_mape_ln"] = train_mape_ln
        dic["test_mape_ln"] = test_mape_ln
        positive_cont.append(dic)
    else:
        negative_cont.append(combination)


    

1


C:\Users\Sarthak\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


2


C:\Users\Sarthak\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


3


...

C:\Users\Sarthak\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


106


C:\Users\Sarthak\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


107


C:\Users\Sarthak\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [347]:
len(positive_cont)

18

In [229]:
#lr_model_ols.summary(xname= list(column_names))

In [348]:
### Create Positive Combination Dataframe
positive_df = pd.DataFrame(positive_cont)
positive_df

combination  marketing_contribution  \
0   [Calender Week, Dec, Mar, Jun, Jul, Christmas ...               35.917973   
1   [Calender Week, Dec, Mar, Jun, Jul, Christmas ...               36.225975   
2   [Calender Week, Dec, Mar, Jun, Jul, Christmas ...               37.399316   
3   [Calender Week, Dec, Mar, Jun, Jul, Christmas ...               48.404148   
4   [Calender Week, Dec, Mar, Jun, Jul, Christmas ...               35.666772   
5   [Calender Week, Dec, Mar, Jun, Jul, 1st week, ...               36.778381   
6   [Calender Week, Dec, Mar, Jun, Christmas Flag,...               53.123258   
7   [Calender Week, Dec, Mar, Jun, Christmas Flag,...               36.080432   
8   [Calender Week, Dec, Mar, Jul, Christmas Flag,...               36.410439   
9   [Calender Week, Dec, Mar, Jul, Christmas Flag,...               36.774347   
10  [Calender Week, Dec, Mar, Jul, Christmas Flag,...               50.814396   
11  [Calender Week, Dec, Mar, Jul, Christmas Flag,...               38.118664   
12  [Calender Week, Dec, Mar, Jul, Christmas Flag,...               35.416930   
13  [Calender Week, Dec, Mar, Jul, 1st week, 2nd w...               37.416852   
14  [Calender Week, Mar, Jun, Christmas Flag, Than...               46.340288   
15  [Calender Week, Mar, Jul, Christmas Flag, Than...               36.288317   
16  [Calender Week, Dec, Mar, Jun, Jul, Christmas ...               35.690554   
17  [Calender Week, Dec, Mar, Jul, Christmas Flag,...               36.438339   

    others_contribution  test_mape  test_mape_ln  train_mape  train_mape_ln  
0             64.082027  38.935864      4.380871   54.647952       5.715212  
1             63.774025  38.490864      4.326621   54.786603       5.728472  
2             62.600684  39.640414      4.492305   54.796148       5.792267  
3             51.595852  42.302933      4.509132   56.099469       5.826373  
4             64.333228  38.492616      4.339974   54.740172       5.724312  
5             63.221619  39.011242      4.410204   56.088069       5.835117  
6             46.876742  42.751188      4.532121   56.511096       5.878146  
7             63.919568  38.951768      4.350668   54.765962       5.737066  
8             63.589561  38.528759      4.328694   54.962068       5.759933  
9             63.225653  38.370184      4.318856   55.047859       5.764096  
10            49.185604  43.783976      4.577398   56.905537       5.877610  
11            61.881336  39.580903      4.471153   55.022172       5.823380  
12            64.583070  39.724159      4.389602   55.223492       5.761586  
13            62.583148  38.319740      4.332314   56.545057       5.888967  
14            53.659712  48.818487      4.887552   57.191550       5.832663  
15            63.711683  45.496698      4.782157   56.810049       5.796888  
16            64.309446  39.052049      4.388248   54.633707       5.716613  
17            63.561661  38.388319      4.309700   54.969289       5.761067

In [349]:
## Sort the values based on marketing contribution - ascending order
positive_df = positive_df.sort_values(['test_mape'])
positive_df.to_excel(path+"\\iterations\\iter_display_D11S15_equally_divided1.xlsx")
positive_df
#positive_df.loc[321,"combination"]

combination  marketing_contribution  \
13  [Calender Week, Dec, Mar, Jul, 1st week, 2nd w...               37.416852   
9   [Calender Week, Dec, Mar, Jul, Christmas Flag,...               36.774347   
17  [Calender Week, Dec, Mar, Jul, Christmas Flag,...               36.438339   
1   [Calender Week, Dec, Mar, Jun, Jul, Christmas ...               36.225975   
4   [Calender Week, Dec, Mar, Jun, Jul, Christmas ...               35.666772   
8   [Calender Week, Dec, Mar, Jul, Christmas Flag,...               36.410439   
0   [Calender Week, Dec, Mar, Jun, Jul, Christmas ...               35.917973   
7   [Calender Week, Dec, Mar, Jun, Christmas Flag,...               36.080432   
5   [Calender Week, Dec, Mar, Jun, Jul, 1st week, ...               36.778381   
16  [Calender Week, Dec, Mar, Jun, Jul, Christmas ...               35.690554   
11  [Calender Week, Dec, Mar, Jul, Christmas Flag,...               38.118664   
2   [Calender Week, Dec, Mar, Jun, Jul, Christmas ...               37.399316   
12  [Calender Week, Dec, Mar, Jul, Christmas Flag,...               35.416930   
3   [Calender Week, Dec, Mar, Jun, Jul, Christmas ...               48.404148   
6   [Calender Week, Dec, Mar, Jun, Christmas Flag,...               53.123258   
10  [Calender Week, Dec, Mar, Jul, Christmas Flag,...               50.814396   
15  [Calender Week, Mar, Jul, Christmas Flag, Than...               36.288317   
14  [Calender Week, Mar, Jun, Christmas Flag, Than...               46.340288   

    others_contribution  test_mape  test_mape_ln  train_mape  train_mape_ln  
13            62.583148  38.319740      4.332314   56.545057       5.888967  
9             63.225653  38.370184      4.318856   55.047859       5.764096  
17            63.561661  38.388319      4.309700   54.969289       5.761067  
1             63.774025  38.490864      4.326621   54.786603       5.728472  
4             64.333228  38.492616      4.339974   54.740172       5.724312  
8             63.589561  38.528759      4.328694   54.962068       5.759933  
0             64.082027  38.935864      4.380871   54.647952       5.715212  
7             63.919568  38.951768      4.350668   54.765962       5.737066  
5             63.221619  39.011242      4.410204   56.088069       5.835117  
16            64.309446  39.052049      4.388248   54.633707       5.716613  
11            61.881336  39.580903      4.471153   55.022172       5.823380  
2             62.600684  39.640414      4.492305   54.796148       5.792267  
12            64.583070  39.724159      4.389602   55.223492       5.761586  
3             51.595852  42.302933      4.509132   56.099469       5.826373  
6             46.876742  42.751188      4.532121   56.511096       5.878146  
10            49.185604  43.783976      4.577398   56.905537       5.877610  
15            63.711683  45.496698      4.782157   56.810049       5.796888  
14            53.659712  48.818487      4.887552   57.191550       5.832663

In [63]:
### Select those combinations which have Week number as one variable. We can modify it to select based on our requirements.
def contains_Element(row,col,element_list):
    if(all(elem in row[col] for elem in element_list)):
        return 1
    else:
        return 0
positive_df["Element"] = positive_df.apply(lambda row:contains_Element(row,"combination",["Week Number"]),axis=1)

In [64]:
positive_df[positive_df.Element==1]
# .sort_values(by=["test_mape"])

combination  marketing_contribution  \
1  [Week Ending, Title, ln_Total Units, Sales Tre...               41.774185   
0  [Week Ending, Title, ln_Total Units, Sales Tre...               48.813331   
2  [Week Ending, Title, ln_Total Units, Sales Tre...               42.364612   
6  [Week Ending, Title, ln_Total Units, Sales Tre...               42.420902   

   others_contribution   test_mape  test_mape_ln  train_mape  train_mape_ln  \
1            58.225815   98.526782      8.627409  106.692672       7.960187   
0            51.186669  100.128291      8.625289  104.781811       8.058588   
2            57.635388  100.313681      8.741725  105.773385       7.914377   
6            57.579098  100.795082      8.759110  105.811848       7.926899   

   Element  
1        1  
0        1  
2        1  
6        1

In [240]:
positive_df.loc[1,"combination"]

['Week Ending',
 'Title',
 'ln_Total Units',
 'Week Number',
 'Thanksgiving Flag',
 '1st week',
 '2nd week',
 'Is Action',
 'Is Open World',
 'same genre',
 'different genre',
 'black Friday week number']

In [256]:
positive_cont

[{'combination': ['Week Ending',
   'Title',
   'ln_Total Units',
   'Sales Trend',
   'Week Number',
   'Meta Critic Score',
   '1st week',
   '2nd week',
   'Christmas Flag',
   'Is Adventure',
   'Is Open World',
   'Thanksgiving Flag',
   'same genre'],
  'marketing_contribution': 48.586440055805845,
  'others_contribution': 51.41355994419414},
 {'combination': ['Week Ending',
   'Title',
   'ln_Total Units',
   'Sales Trend',
   'Week Number',
   'Meta Critic Score',
   '1st week',
   '2nd week',
   'Christmas Flag',
   'Is Adventure',
   'Is Open World',
   'Thanksgiving Flag',
   'different genre'],
  'marketing_contribution': 48.93238897312431,
  'others_contribution': 51.067611026875696},
 {'combination': ['Week Ending',
   'Title',
   'ln_Total Units',
   'Sales Trend',
   'Week Number',
   'Meta Critic Score',
   '1st week',
   '2nd week',
   'Christmas Flag',
   'Is Adventure',
   'Is Open World',
   'Thanksgiving Flag',
   'black Friday week number'],
  'marketing_contribu

In [49]:
len(all_combinations)

32768

In [121]:
df_marketing_percent_ln.to_excel(r"D:\Affine_Projects\WB_Warner_Bros\MMM\AD_Creation\Lego_AD\19062019\marketing_adstock_ln.xlsx")

### NNLS/Lasso

In [1207]:
### Lasso model / nnls  to get all positive coef
from sklearn import linear_model
from scipy.optimize import nnls

df_lasso = all_train_dfs[27]
dict_preprocess_data = getPreProcessData(df_lasso)
XTrainScaledConst = dict_preprocess_data['XTrainScaledConst']
XTestScaledConst = dict_preprocess_data['XTestScaledConst']
X_train_all = dict_preprocess_data['X_train_all']
X_test_all = dict_preprocess_data['X_test_all']
Y_train = dict_preprocess_data['Y_train']
Y_test = dict_preprocess_data['Y_test']
scaler = dict_preprocess_data['scaler']
XTrainScaledConst =  XTrainScaledConst.drop(['const'],axis=1)
#x,rnorm = nnls(XTrainScaledConst.values,Y_train.values)

lasso = linear_model.Lasso(alpha=.0001,positive=True,random_state=7,selection='random')
lasso.fit(XTrainScaledConst,Y_train)

C:\Users\Sarthak\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Sarthak\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
C:\Users\Sarthak\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.


Lasso(alpha=0.0001, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=True, precompute=False, random_state=7,
   selection='random', tol=0.0001, warm_start=False)

In [436]:
x

array([0.        , 0.85857489, 0.65023462, 0.        , 0.        ,
       0.05110996, 0.        , 0.        , 0.        , 1.22460368,
       0.        , 0.0526936 ])

In [437]:
rnorm 

150.96839480255613

In [1045]:
print(lasso.intercept_)
print(lasso.coef_)

7.576724056341367
[0.         0.46682802 0.66308008 0.         0.         0.
 0.19680184 0.         0.19961848 0.         0.09254818 0.01429033
 0.         0.17217909 0.19468733 0.31397099 0.82163344 0.
 0.        ]


In [71]:
#lr_model_ols = df_model_params[85]["lr_model_ols"]
#col_name = df_model_params[85]["column_names"]
#lr_model_ols.summary(xname = list(col_name))

In [1300]:
df_vif = df_model_params[27]["XTrainScaledConst"]
## ,'Install base','Is Other','Week Number','Is Fighting','Meta Critic Score'
df_vif = df_vif.drop(['const'],axis=1)
vif = variance_inflation_factors(df_vif)
vif = vif.reset_index()
vif = vif.sort_values(by=['VIF'],ascending=0)
vif

index       VIF
19        ln_ads_OtherTradeSpend_D3_S6  6.220012
17              ln_ads_POSRebate_D2_S5  5.483525
2                          Week Number  4.843277
3                    Meta Critic Score  4.643744
1                                  ASP  4.058175
13                        TubularViews  3.741847
16                ln_ads_Display_D3_S5  3.463400
14                ln_ads_TVSpend_D1_S7  2.949377
15       ln_ads_VideoImpressions_D3_S5  2.706133
18  ln_ads_TradeDisplayPlacement_D1_S6  2.552520
8                         Is Adventure  2.468112
7                            Is Action  1.959011
10                   Thanksgiving Flag  1.692288
12            black Friday week number  1.618023
9                        Is Open World  1.449987
5                             2nd week  1.433072
4                             1st week  1.291018
11                     different genre  1.089972
6                       Christmas Flag  1.065163
0                                const  1.000000

In [1301]:
### For Calculating Each Contribution of the selected model
model_selected_num = 27
model_dict = df_model_params[model_selected_num]
lr_model_ols= model_dict["lr_model_ols"]
column_names= model_dict["column_names"]
XTrainScaledConst = model_dict["XTrainScaledConst"]
XTestScaledConst = model_dict["XTestScaledConst"]
X_train_all = model_dict["X_train_all"]
X_test_all = model_dict["X_test_all"]
Y_train = model_dict["Y_train"]
Y_test = model_dict["Y_test"]
scaler = model_dict["scaler"]
betaCoef = lr_model_ols.params
ypred = lr_model_ols.predict(XTrainScaledConst)
ypred_exp = np.exp(ypred.values)

In [1302]:
df_actual_predicted_train = X_train_all.copy()
df_actual_predicted_test = X_test_all.copy()
df_actual_predicted_train['Y_actual_ln'] = Y_train
df_actual_predicted_train['Y_predicted_ln'] = ypred.values
df_actual_predicted_test['Y_actual_ln'] = Y_test
df_actual_predicted_test['Y_predicted_ln'] = lr_model_ols.predict(XTestScaledConst).values


In [1157]:
#df_actual_predicted_train

In [1158]:
#df_actual_predicted_test

In [1159]:
#df_actual_predicted_train.to_excel(r"D:\Affine_Projects\WB_Warner_Bros\MMM\AD_Creation\adstock\adstock_12062019\contributions\train_actual_predicted_192.xlsx")
#df_actual_predicted_test.to_excel(r"D:\Affine_Projects\WB_Warner_Bros\MMM\AD_Creation\adstock\adstock_12062019\contributions\test_actual_predicted_192.xlsx")


In [38]:
raw_contribution = getRawContributions(XTrainScaledConst,ypred_exp,betaCoef,column_names)
scaled_contribution = getScaledContributions(XTrainScaledConst,ypred_exp,raw_contribution,column_names)
scaled_contribution_df = pd.DataFrame(scaled_contribution,columns = column_names)
grouped,total = getPercentMarketingContribution(scaled_contribution_df,16)

In [39]:
total

variables         value Marketing_Variables
0                                const  2.708004e+06              others
1                                  ASP -1.555496e+06              others
2                          Sales Trend  4.233410e+05              others
3                    Meta Critic Score -1.038806e+05              others
4                          Week Number  1.613399e+05              others
5                             1st week  4.806114e+04              others
6                             2nd week  3.807155e+04              others
7                       Christmas Flag  8.359120e+04              others
8                            Is Action  2.951598e+04              others
9                         Is Adventure -4.095068e+00              others
10                       Is Open World -4.725398e+04              others
11                   Thanksgiving Flag -1.158480e+04              others
12                          same genre -6.685686e+03              others
13                     different genre  2.189197e+03              others
14            black Friday week number  1.683592e+05              others
15                        TubularViews  1.609244e+05              others
16                ln_ads_TVSpend_D1_S8  1.550383e+05           marketing
17               ln_ads_Facebook_D3_S5  3.583913e+04           marketing
18                  ln_ads_Video_D3_S4  1.747099e+05           marketing
19                ln_ads_Display_D3_S5  1.886143e+05           marketing
20                 ln_ads_Search_D3_S4  2.488510e+04           marketing
21              ln_ads_POSRebate_D1_S5 -5.577576e+04           marketing
22  ln_ads_TradeDisplayPlacement_D1_S5  3.737811e+05           marketing

In [40]:
grouped

Marketing_Variables         value    percent
0           marketing  8.970922e+05  29.947151
1              others  2.098492e+06  70.052849

In [159]:
## scaled contribution consists of the contribution for each variable.
#scaled_contribution[0]

In [160]:
## To Create scaled_contribution dataframe
# scaled_contribution_df = pd.DataFrame(scaled_contribution,columns = column_names)
# scaled_contribution_df

In [161]:
### Write Scaled_Contributions to excel
#scaled_contribution_df.to_excel(r"D:\Affine_Projects\WB_Warner_Bros\MMM\AD_Creation\adstock\adstock_12062019\contributions\contribution_192.xlsx")

In [1517]:
## Since Predicted Value and our Contribution is coming to be same, so the process is correct.
print(ypred_exp[0])
print(np.sum(scaled_contribution[0]))

4952.760986270104
4952.760986270103


# PCA  

In [242]:
## For PCA
df_selected_pca =  all_train_dfs[23].copy()
pca_cols = ['ln_Facebook','ln_Video','ln_ads_Search_D3_S5','ln_Display']
nonpca_cols = np.setdiff1d(df_selected_pca.columns,pca_cols,assume_unique=True)
nonpca_cols = [x for x in nonpca_cols if x not in ['Week Ending','Title','Total Revenue','ln_Total Units']]

### Standardised Data
dict_preprocess_data = getPreProcessData(df_selected_pca)
XTrainScaledConst = dict_preprocess_data['XTrainScaledConst']
XTrainScaledConst =  XTrainScaledConst.drop(['const'],axis=1)
XTestScaledConst = dict_preprocess_data['XTestScaledConst']
XTestScaledConst = XTestScaledConst.drop(['const'],axis=1)
X_train_all = dict_preprocess_data['X_train_all']
X_test_all = dict_preprocess_data['X_test_all']
Y_train = dict_preprocess_data['Y_train']
Y_test = dict_preprocess_data['Y_test']
scaler = dict_preprocess_data['scaler']
print(XTrainScaledConst.columns)
### Creating PCA train 
df_pca_train = XTrainScaledConst[pca_cols]
df_pca_test = XTestScaledConst[pca_cols]
df_nonpca_train = XTrainScaledConst[nonpca_cols]
df_nonpca_test = XTestScaledConst[nonpca_cols]

Index(['ASP', 'Sales Trend', 'Metacritic number of user ratings', 'Is Action',
       'Is Adventure', 'Is Open World', 'Thanksgiving Flag', 'Christmas Flag',
       '1st week', '2nd week', 'black Friday week number', 'same genre',
       'different genre', 'TubularViews', 'ln_TVSpend', 'ln_Facebook',
       'ln_Video', 'ln_ads_Search_D3_S5', 'ln_Display',
       'ln_ads_POSRebate_D1_S8', 'ln_ads_TradeDisplayPlacement_D1_S5',
       'ln_ads_OtherTradeSpend_D3_S5'],
      dtype='object')


C:\Users\Sarthak\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Sarthak\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
C:\Users\Sarthak\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.


In [243]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(df_pca_train)
print("Explained_variance_ratio: ",pca.explained_variance_ratio_)
pca_transformed_train_array = pca.transform(df_pca_train)
pca_transformed_test_array = pca.transform(df_pca_test)
eigenvectors = pca.components_
eigenvalues = pca.explained_variance_
print("EigenVectors: ")
print(eigenvectors)
pca_transformed_train_df = pd.DataFrame(pca_transformed_train_array,columns = ['pca1','pca2'])
pca_transformed_test_df = pd.DataFrame(pca_transformed_test_array,columns = ['pca1','pca2'])
final_train_df = pd.concat([df_nonpca_train,pca_transformed_train_df],axis=1)
final_test_df = pd.concat([df_nonpca_test,pca_transformed_test_df],axis=1)
final_train_df = sm.add_constant(final_train_df)
final_test_df = sm.add_constant(final_test_df,has_constant='add')
final_train_df.head()

Explained_variance_ratio:  [0.62437511 0.22804627]
EigenVectors: 
[[ 0.50631322  0.5954314   0.23584096  0.57748368]
 [ 0.14745811  0.13835871 -0.97136449  0.12475577]]


const       ASP  Sales Trend  Metacritic number of user ratings  Is Action  \
0    1.0  0.804674    -0.006551                          -0.153677   0.369096   
1    1.0  0.979660     0.992499                          -0.296092  -2.709322   
2    1.0 -1.241231    -0.870337                          -1.150580   0.369096   
3    1.0 -0.435183    -0.489880                           1.341677   0.369096   
4    1.0 -0.053383     1.484688                          -1.577823   0.369096   

   Is Adventure  Is Open World  Thanksgiving Flag  Christmas Flag  1st week  \
0      0.804400      -0.377964          -0.177705        -0.19245 -0.124676   
1     -1.243163      -0.377964          -0.177705        -0.19245 -0.124676   
2      0.804400      -0.377964          -0.177705        -0.19245 -0.124676   
3      0.804400      -0.377964          -0.177705        -0.19245 -0.124676   
4     -1.243163      -0.377964          -0.177705        -0.19245 -0.124676   

     ...     black Friday week number  same genre  different genre  \
0    ...                    -0.103489   -0.071611        -0.410497   
1    ...                    -0.103489   -0.071611         2.184903   
2    ...                    -0.103489   -0.071611        -0.410497   
3    ...                    -0.103489   -0.071611        -0.410497   
4    ...                    -0.103489   -0.071611         2.184903   

   TubularViews  ln_TVSpend  ln_ads_POSRebate_D1_S8  \
0      0.453948   -0.281187                0.926385   
1      0.292015   -0.281187                0.423562   
2     -0.648385   -0.281187               -1.501430   
3     -0.648385   -0.281187                0.789975   
4     -0.194677   -0.281187                0.663202   

   ln_ads_TradeDisplayPlacement_D1_S5  ln_ads_OtherTradeSpend_D3_S5      pca1  \
0                            0.475491                      1.084904 -0.594045   
1                            0.554652                      0.194870 -0.636852   
2                            0.011536                     -1.780152 -0.636852   
3                            0.471466                      0.323252 -0.636852   
4                            0.338279                      0.707007 -0.601545   

       pca2  
0 -0.718567  
1 -0.542256  
2 -0.542256  
3 -0.542256  
4 -0.687676  

[5 rows x 21 columns]

In [244]:
lr_model_ols_pca,column_names = getLinearModel(final_train_df,Y_train)
lr_model_ols_pca.summary(xname= list(column_names))

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.890
Model:                            OLS   Adj. R-squared:                  0.884
Method:                 Least Squares   F-statistic:                     150.6
Date:                Mon, 17 Jun 2019   Prob (F-statistic):          2.80e-164
Time:                        14:03:24   Log-Likelihood:                -409.39
No. Observations:                 392   AIC:                             860.8
Df Residuals:                     371   BIC:                             944.2
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
const                                  7.5767      0.036    212.246      0.000       7.507       7.647
ASP                                   -0.5695      0.049    -11.518      0.000      -0.667      -0.472
Sales Trend                            0.9060      0.079     11.457      0.000       0.751       1.062
Metacritic number of user ratings      0.6556      0.042     15.774      0.000       0.574       0.737
Is Action                             -0.1223      0.044     -2.792      0.006      -0.208      -0.036
Is Adventure                          -0.0074      0.056     -0.133      0.895      -0.118       0.103
Is Open World                         -0.0856      0.043     -1.992      0.047      -0.170      -0.001
Thanksgiving Flag                      0.0392      0.047      0.838      0.403      -0.053       0.131
Christmas Flag                         0.1540      0.037      4.171      0.000       0.081       0.227
1st week                              -0.0350      0.045     -0.774      0.440      -0.124       0.054
2nd week                              -0.0195      0.045     -0.436      0.663      -0.108       0.069
black Friday week number               0.1845      0.046      4.036      0.000       0.095       0.274
same genre                             0.0313      0.037      0.858      0.392      -0.040       0.103
different genre                        0.0298      0.037      0.804      0.422      -0.043       0.103
TubularViews                           0.0395      0.058      0.678      0.498      -0.075       0.154
ln_TVSpend                             0.0575      0.068      0.851      0.395      -0.075       0.190
ln_ads_POSRebate_D1_S8                 0.7026      0.074      9.499      0.000       0.557       0.848
ln_ads_TradeDisplayPlacement_D1_S5     0.0347      0.052      0.669      0.504      -0.067       0.137
ln_ads_OtherTradeSpend_D3_S5           0.6758      0.074      9.108      0.000       0.530       0.822
pca1                                   0.0038      0.033      0.116      0.908      -0.060       0.068
pca2                                  -0.0263      0.051     -0.520      0.603      -0.126       0.073
==============================================================================
Omnibus:                       23.977   Durbin-Watson:                   1.982
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               78.511
Skew:                          -0.040   Prob(JB):                     8.95e-18
Kurtosis:                       5.191   Cond. No.                         6.30
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [245]:
ypredict_train_pca = lr_model_ols_pca.predict(final_train_df)
ypredict_test_pca = lr_model_ols_pca.predict(final_test_df)
train_mape_pca = mean_absolute_percentage_error(Y_train,ypredict_train.values)
test_mape_pca = mean_absolute_percentage_error(Y_test,ypredict_test.values)
ypred_exp_train_pca = np.exp(ypredict_train_pca.values)
ypred_exp_test_pca = np.exp(ypredict_test_pca.values)
pca_betacoef = lr_model_ols_pca.params
cols = final_train_df.columns
print(train_mape_pca)
print(test_mape_pca)

7.425992330139564
8.733333900286034


In [246]:
## Convert equation to original variables
pca_var_betaCoef = pca_betacoef[-2:]
pca_var_betaCoef = pca_var_betaCoef.reshape((pca_var_betaCoef.shape[0],1))
print(pca_var_betaCoef)
print(pca_var_betaCoef.shape)
eigenvectors = pca.components_
print(eigenvectors)
print(eigenvectors.shape)
betacoef_var_considered_for_pca  =  np.transpose(pca_var_betaCoef) @ eigenvectors
print(betacoef_var_considered_for_pca)
print(betacoef_var_considered_for_pca.shape)

[[ 0.00378691]
 [-0.02629207]]
(2, 1)
[[ 0.50631322  0.5954314   0.23584096  0.57748368]
 [ 0.14745811  0.13835871 -0.97136449  0.12475577]]
(2, 4)
[[-0.00195962 -0.00138289  0.02643229 -0.00109321]]
(1, 4)


In [247]:
## VIF on PCA
df_vif = final_train_df
df_vif = df_vif.drop(['const'],axis=1)
vif = variance_inflation_factors(df_vif)
vif = vif.reset_index()
vif = vif.sort_values(by=['VIF'],ascending=0)
vif

index       VIF
2                          Sales Trend  4.907625
18        ln_ads_OtherTradeSpend_D3_S5  4.319954
16              ln_ads_POSRebate_D1_S8  4.292830
15                          ln_TVSpend  3.587952
14                        TubularViews  2.664504
5                         Is Adventure  2.472540
17  ln_ads_TradeDisplayPlacement_D1_S5  2.113456
19                                pca1  2.094032
1                                  ASP  1.918635
20                                pca2  1.828695
7                    Thanksgiving Flag  1.716656
11            black Friday week number  1.639637
9                             1st week  1.605892
10                            2nd week  1.572799
4                            Is Action  1.504831
6                        Is Open World  1.447286
3    Metacritic number of user ratings  1.355449
13                     different genre  1.080112
8                       Christmas Flag  1.068870
12                          same genre  1.046657
0                                const  1.000000

In [248]:
raw_contributions_pca = getRawContributions(final_train_df,ypred_exp_train_pca,pca_betacoef,cols)
scaled_contributions_pca = getScaledContributions(final_train_df,ypred_exp_train_pca,raw_contributions_pca,cols)
scaled_contribution_df = pd.DataFrame(scaled_contributions_pca,columns = cols)
grouped_pca,total_pca = getPercentMarketingContribution(scaled_contribution_df,15)

In [249]:
total_pca

variables         value Marketing_Variables
0                                const  2.085712e+06              others
1                                  ASP -7.251955e+05              others
2                          Sales Trend  7.552708e+05              others
3    Metacritic number of user ratings  1.060076e+05              others
4                            Is Action -1.098475e+05              others
5                         Is Adventure -6.779665e+03              others
6                        Is Open World -5.580388e+04              others
7                    Thanksgiving Flag  2.974062e+04              others
8                       Christmas Flag -4.877704e+04              others
9                             1st week -5.919883e+04              others
10                            2nd week -4.902053e+04              others
11            black Friday week number  2.211748e+04              others
12                          same genre -6.454038e+03              others
13                     different genre -2.349068e+04              others
14                        TubularViews  3.384583e+04              others
15                          ln_TVSpend  9.277129e+04           marketing
16              ln_ads_POSRebate_D1_S8  5.179458e+05           marketing
17  ln_ads_TradeDisplayPlacement_D1_S5  2.387641e+04           marketing
18        ln_ads_OtherTradeSpend_D3_S5  5.254433e+05           marketing
19                                pca1  8.048035e+03           marketing
20                                pca2 -1.381081e+04           marketing

In [250]:
grouped_pca

Marketing_Variables         value   percent
0           marketing  1.154274e+06  37.20583
1              others  1.948127e+06  62.79417

In [77]:
### Writing the training data to repeat the same contribution process in excel.
# X_AD = XTrainScaledConst.copy() 
# X_AD["y"] = ypred_exp 
# X_AD.to_excel("D:\\Affine_Projects\\WB_Warner_Bros\\MMM\\AD_Creation\\adstock\\adstock_10062019\\XTrain_scaled.xlsx")
# beta_coef_df = pd.DataFrame(columns=['coef'])
# beta_coef_df['coef'] =  betaCoef
# beta_coef_df.to_excel("D:\\Affine_Projects\\WB_Warner_Bros\\MMM\\AD_Creation\\adstock\\adstock_10062019\\beta_coef.xlsx")